初期設定

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!mkdir /root/.kaggle 
!cp drive/My\ Drive/kaggle.json /root/.kaggle
!pip install kaggle

データ初期化

In [0]:
import os
import subprocess
import shutil
import pandas as pd
import glob

CLASS_COUNT_LIMIT = 8
VALIDATION_SPLIT = 0.1
SEED = 1470
WITHOUT_NEW_WHALE = True
ONLY_NEW_WHALE = False

BASE_DIR = 'data'
TRAIN_DIR = os.path.join(BASE_DIR, 'train')
VALIDATION_DIR = os.path.join(BASE_DIR, 'validation')

def download(clean=False):
    if clean:
       if os.path.exists('data'):
          shutil.rmtree('data')
    if not os.path.exists('data'):
        os.mkdir('data')
    os.chdir('data')
    if not os.path.exists('train.zip'):
        subprocess.call(['kaggle', 'competitions', 'download', '-c', 'humpback-whale-identification'])
    if os.path.exists('test'):
        shutil.rmtree('test')
    os.mkdir('test')
    os.chdir('test')
    subprocess.call(['unzip', '../test.zip', '-d', '0'])
    os.chdir('../')
    if os.path.exists('train'):
        shutil.rmtree('train')
    subprocess.call(['unzip', 'train.zip', '-d', 'train'])
    if os.path.exists('validation'):
        shutil.rmtree('validation')
    os.mkdir('validation')
    os.chdir('../')

def train_validation_split(class_list, validation_split):
    # 5class 未満は全てtrain
    class_count = class_list.count()[0]
    if class_count < CLASS_COUNT_LIMIT:
        return class_list, pd.DataFrame(columns=class_list.columns)
    # 1 or n * validation_split
    validation_count = int(max(class_count * validation_split, 1))
    train_count = class_count - validation_count
    validation_list = class_list.sample(validation_count, random_state=SEED)
    train_list = class_list.drop(validation_list.index)
    return train_list, validation_list

def initialize_train_validation(clean = False):
    # make train and validation data
    # nc = nc > 5 && nc else 0 
    # validation_count = max( 1, nc/10 )
    classes = pd.read_csv('data/train.csv')
    class_value_counts = classes['Id'].value_counts().reset_index()
    # 
    if clean:
        path = os.path.join('./' + TRAIN_DIR, '**/*.txt')
        [shutil.move(x, TRAIN_DIR) for x in glob.glob(path, recursive=True)]
        path = os.path.join('./' + VALIDATION_DIR, '**/*.txt')
        [shutil.move(x, TRAIN_DIR) for x in glob.glob(path, recursive=True)]
        glob.glob(path, recursive=True)
        [shutil.rmtree(os.path.join(TRAIN_DIR, x)) for x in os.listdir(TRAIN_DIR) if os.path.isdir(os.path.join(TRAIN_DIR, x))]
        [shutil.rmtree(os.path.join(VALIDATION_DIR, x)) for x in os.listdir(VALIDATION_DIR) if os.path.isdir(os.path.join(VALIDATION_DIR, x))]
    if not os.path.exists(TRAIN_DIR):
        os.mkdir(TRAIN_DIR)
    if not os.path.exists(VALIDATION_DIR):
        os.mkdir(VALIDATION_DIR)
    for i, v in class_value_counts.iterrows():
        class_name = v[0]
        class_files_list = classes[classes['Id'] == class_name]
        # define class
        # class known whale and new whale
        if WITHOUT_NEW_WHALE: 
            if class_name == 'new_whale':
                unuse_dir = os.path.join(BASE_DIR, 'unuse')
                os.mkdir(unuse_dir)
                [shutil.move(os.path.join(TRAIN_DIR, x[1]['Image']), unuse_dir) for x in class_files_list.iterrows()]    
                continue
        elif ONLY_NEW_WHALE:
            if class_name != 'new_whale':
                class_name = 'known_whale'

        train_output_dir = os.path.join(TRAIN_DIR, class_name)
        if not os.path.exists(train_output_dir):
            os.mkdir(train_output_dir)
        validation_output_dir = os.path.join(VALIDATION_DIR, class_name)
        if not os.path.exists(validation_output_dir):
            os.mkdir(validation_output_dir)
        train_list, validation_list = train_validation_split(class_files_list, VALIDATION_SPLIT)
        [shutil.move(os.path.join(TRAIN_DIR, x[1]['Image']), validation_output_dir) for x in validation_list.iterrows()]
        [shutil.move(os.path.join(TRAIN_DIR, x[1]['Image']), train_output_dir) for x in train_list.iterrows()]


download(True)
initialize_train_validation(True)

assert(len(os.listdir('./data/validation')) == len(os.listdir('./data/train')))


In [4]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import resnet50
from keras import models
from keras import layers
from keras import optimizers
from keras.engine.topology import Input
from keras.callbacks import ModelCheckpoint, CSVLogger, TensorBoard

import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

base_dir='data'
train_dir=os.path.join(base_dir, 'train')
validation_dir=os.path.join(base_dir, 'validation')
model_base_dir = 'models/without_whale'

# paramater
batch_size=32
SEED=1470
epochs=50

input_size = 224

def load_data():
    datagen = ImageDataGenerator(
        rescale=1./255,
        horizontal_flip=True,
        rotation_range=30,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        brightness_range=[0.7, 1.0],
    )
    train_generator = datagen.flow_from_directory(
        train_dir,
        target_size=(input_size, input_size),
        batch_size=batch_size,
        seed=SEED,
        class_mode='categorical',
    )
    datagen = ImageDataGenerator(
        rescale=1./255,
    )
    val_generator = datagen.flow_from_directory(
        validation_dir,
        target_size=(input_size, input_size),
        batch_size=20,
        seed=SEED,
        class_mode='categorical',
    )
    return train_generator, val_generator


Using TensorFlow backend.


In [0]:
# Epoch 115/120
# 231/231 [==============================] - 421s 2s/step - loss: 4.9588 - acc: 0.4170 - val_loss: 4.3598 - val_acc: 0.5015
# 5004 class
# Epoch 00115: val_acc improved from 0.49243 to 0.50151, saving model to drive/My Drive/models/cnn_model1/115-4.96-4.36.hdf5

def get_model():
    conv_base = resnet50.ResNet50(weights='imagenet',
                      include_top=False,
                      input_shape=(input_size, input_size, 3))
    for layer in conv_base.layers[:139]: # default 179
       if 'BatchNormalization' in str(layer):
          layer.trainable = True
       else:
         layer.trainable = False
    model = models.Sequential()
    model.add(conv_base)
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dense(1024, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.BatchNormalization())
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.BatchNormalization())
    model.add(layers.Dense(5004))
    model.add(layers.Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=optimizers.RMSprop(lr=1e-4), metrics=['acc'])
    return model


In [0]:
def get_model():
    conv_base = resnet50.ResNet50(weights='imagenet',
                      include_top=False,
                      input_shape=(input_size, input_size, 3))
    for layer in conv_base.layers[:]: # default 179
        if 'BatchNormalization' in str(layer):
            layer.trainable = True
        else:
            layer.trainable = False
    main_input = conv_base.input
    embedding = conv_base.output
    x = layers.GlobalAveragePooling2D()(embedding)
    x = layers.Dense(4096, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(5004, activation='softmax')(x)
    model = models.Model(inputs=[main_input], outputs=[x])
    model.compile(
        loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
    return model




```
Found 15160 images belonging to 5004 classes.
Found 537 images belonging to 5004 classes.
/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '
WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
==================================================================================================
input_14 (InputLayer)           (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_14[0][0]                   
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation_638 (Activation)     (None, 112, 112, 64) 0           bn_conv1[0][0]                   
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 114, 114, 64) 0           activation_638[0][0]             
__________________________________________________________________________________________________
max_pooling2d_14 (MaxPooling2D) (None, 56, 56, 64)   0           pool1_pad[0][0]                  
__________________________________________________________________________________________________
res2a_branch2a (Conv2D)         (None, 56, 56, 64)   4160        max_pooling2d_14[0][0]           
__________________________________________________________________________________________________
bn2a_branch2a (BatchNormalizati (None, 56, 56, 64)   256         res2a_branch2a[0][0]             
__________________________________________________________________________________________________
activation_639 (Activation)     (None, 56, 56, 64)   0           bn2a_branch2a[0][0]              
__________________________________________________________________________________________________
res2a_branch2b (Conv2D)         (None, 56, 56, 64)   36928       activation_639[0][0]             
__________________________________________________________________________________________________
bn2a_branch2b (BatchNormalizati (None, 56, 56, 64)   256         res2a_branch2b[0][0]             
__________________________________________________________________________________________________
activation_640 (Activation)     (None, 56, 56, 64)   0           bn2a_branch2b[0][0]              
__________________________________________________________________________________________________
res2a_branch2c (Conv2D)         (None, 56, 56, 256)  16640       activation_640[0][0]             
__________________________________________________________________________________________________
res2a_branch1 (Conv2D)          (None, 56, 56, 256)  16640       max_pooling2d_14[0][0]           
__________________________________________________________________________________________________
bn2a_branch2c (BatchNormalizati (None, 56, 56, 256)  1024        res2a_branch2c[0][0]             
__________________________________________________________________________________________________
bn2a_branch1 (BatchNormalizatio (None, 56, 56, 256)  1024        res2a_branch1[0][0]              
__________________________________________________________________________________________________
add_209 (Add)                   (None, 56, 56, 256)  0           bn2a_branch2c[0][0]              
                                                                 bn2a_branch1[0][0]               
__________________________________________________________________________________________________
activation_641 (Activation)     (None, 56, 56, 256)  0           add_209[0][0]                    
__________________________________________________________________________________________________
res2b_branch2a (Conv2D)         (None, 56, 56, 64)   16448       activation_641[0][0]             
__________________________________________________________________________________________________
bn2b_branch2a (BatchNormalizati (None, 56, 56, 64)   256         res2b_branch2a[0][0]             
__________________________________________________________________________________________________
activation_642 (Activation)     (None, 56, 56, 64)   0           bn2b_branch2a[0][0]              
__________________________________________________________________________________________________
res2b_branch2b (Conv2D)         (None, 56, 56, 64)   36928       activation_642[0][0]             
__________________________________________________________________________________________________
bn2b_branch2b (BatchNormalizati (None, 56, 56, 64)   256         res2b_branch2b[0][0]             
__________________________________________________________________________________________________
activation_643 (Activation)     (None, 56, 56, 64)   0           bn2b_branch2b[0][0]              
__________________________________________________________________________________________________
res2b_branch2c (Conv2D)         (None, 56, 56, 256)  16640       activation_643[0][0]             
__________________________________________________________________________________________________
bn2b_branch2c (BatchNormalizati (None, 56, 56, 256)  1024        res2b_branch2c[0][0]             
__________________________________________________________________________________________________
add_210 (Add)                   (None, 56, 56, 256)  0           bn2b_branch2c[0][0]              
                                                                 activation_641[0][0]             
__________________________________________________________________________________________________
activation_644 (Activation)     (None, 56, 56, 256)  0           add_210[0][0]                    
__________________________________________________________________________________________________
res2c_branch2a (Conv2D)         (None, 56, 56, 64)   16448       activation_644[0][0]             
__________________________________________________________________________________________________
bn2c_branch2a (BatchNormalizati (None, 56, 56, 64)   256         res2c_branch2a[0][0]             
__________________________________________________________________________________________________
activation_645 (Activation)     (None, 56, 56, 64)   0           bn2c_branch2a[0][0]              
__________________________________________________________________________________________________
res2c_branch2b (Conv2D)         (None, 56, 56, 64)   36928       activation_645[0][0]             
__________________________________________________________________________________________________
bn2c_branch2b (BatchNormalizati (None, 56, 56, 64)   256         res2c_branch2b[0][0]             
__________________________________________________________________________________________________
activation_646 (Activation)     (None, 56, 56, 64)   0           bn2c_branch2b[0][0]              
__________________________________________________________________________________________________
res2c_branch2c (Conv2D)         (None, 56, 56, 256)  16640       activation_646[0][0]             
__________________________________________________________________________________________________
bn2c_branch2c (BatchNormalizati (None, 56, 56, 256)  1024        res2c_branch2c[0][0]             
__________________________________________________________________________________________________
add_211 (Add)                   (None, 56, 56, 256)  0           bn2c_branch2c[0][0]              
                                                                 activation_644[0][0]             
__________________________________________________________________________________________________
activation_647 (Activation)     (None, 56, 56, 256)  0           add_211[0][0]                    
__________________________________________________________________________________________________
res3a_branch2a (Conv2D)         (None, 28, 28, 128)  32896       activation_647[0][0]             
__________________________________________________________________________________________________
bn3a_branch2a (BatchNormalizati (None, 28, 28, 128)  512         res3a_branch2a[0][0]             
__________________________________________________________________________________________________
activation_648 (Activation)     (None, 28, 28, 128)  0           bn3a_branch2a[0][0]              
__________________________________________________________________________________________________
res3a_branch2b (Conv2D)         (None, 28, 28, 128)  147584      activation_648[0][0]             
__________________________________________________________________________________________________
bn3a_branch2b (BatchNormalizati (None, 28, 28, 128)  512         res3a_branch2b[0][0]             
__________________________________________________________________________________________________
activation_649 (Activation)     (None, 28, 28, 128)  0           bn3a_branch2b[0][0]              
__________________________________________________________________________________________________
res3a_branch2c (Conv2D)         (None, 28, 28, 512)  66048       activation_649[0][0]             
__________________________________________________________________________________________________
res3a_branch1 (Conv2D)          (None, 28, 28, 512)  131584      activation_647[0][0]             
__________________________________________________________________________________________________
bn3a_branch2c (BatchNormalizati (None, 28, 28, 512)  2048        res3a_branch2c[0][0]             
__________________________________________________________________________________________________
bn3a_branch1 (BatchNormalizatio (None, 28, 28, 512)  2048        res3a_branch1[0][0]              
__________________________________________________________________________________________________
add_212 (Add)                   (None, 28, 28, 512)  0           bn3a_branch2c[0][0]              
                                                                 bn3a_branch1[0][0]               
__________________________________________________________________________________________________
activation_650 (Activation)     (None, 28, 28, 512)  0           add_212[0][0]                    
__________________________________________________________________________________________________
res3b_branch2a (Conv2D)         (None, 28, 28, 128)  65664       activation_650[0][0]             
__________________________________________________________________________________________________
bn3b_branch2a (BatchNormalizati (None, 28, 28, 128)  512         res3b_branch2a[0][0]             
__________________________________________________________________________________________________
activation_651 (Activation)     (None, 28, 28, 128)  0           bn3b_branch2a[0][0]              
__________________________________________________________________________________________________
res3b_branch2b (Conv2D)         (None, 28, 28, 128)  147584      activation_651[0][0]             
__________________________________________________________________________________________________
bn3b_branch2b (BatchNormalizati (None, 28, 28, 128)  512         res3b_branch2b[0][0]             
__________________________________________________________________________________________________
activation_652 (Activation)     (None, 28, 28, 128)  0           bn3b_branch2b[0][0]              
__________________________________________________________________________________________________
res3b_branch2c (Conv2D)         (None, 28, 28, 512)  66048       activation_652[0][0]             
__________________________________________________________________________________________________
bn3b_branch2c (BatchNormalizati (None, 28, 28, 512)  2048        res3b_branch2c[0][0]             
__________________________________________________________________________________________________
add_213 (Add)                   (None, 28, 28, 512)  0           bn3b_branch2c[0][0]              
                                                                 activation_650[0][0]             
__________________________________________________________________________________________________
activation_653 (Activation)     (None, 28, 28, 512)  0           add_213[0][0]                    
__________________________________________________________________________________________________
res3c_branch2a (Conv2D)         (None, 28, 28, 128)  65664       activation_653[0][0]             
__________________________________________________________________________________________________
bn3c_branch2a (BatchNormalizati (None, 28, 28, 128)  512         res3c_branch2a[0][0]             
__________________________________________________________________________________________________
activation_654 (Activation)     (None, 28, 28, 128)  0           bn3c_branch2a[0][0]              
__________________________________________________________________________________________________
res3c_branch2b (Conv2D)         (None, 28, 28, 128)  147584      activation_654[0][0]             
__________________________________________________________________________________________________
bn3c_branch2b (BatchNormalizati (None, 28, 28, 128)  512         res3c_branch2b[0][0]             
__________________________________________________________________________________________________
activation_655 (Activation)     (None, 28, 28, 128)  0           bn3c_branch2b[0][0]              
__________________________________________________________________________________________________
res3c_branch2c (Conv2D)         (None, 28, 28, 512)  66048       activation_655[0][0]             
__________________________________________________________________________________________________
bn3c_branch2c (BatchNormalizati (None, 28, 28, 512)  2048        res3c_branch2c[0][0]             
__________________________________________________________________________________________________
add_214 (Add)                   (None, 28, 28, 512)  0           bn3c_branch2c[0][0]              
                                                                 activation_653[0][0]             
__________________________________________________________________________________________________
activation_656 (Activation)     (None, 28, 28, 512)  0           add_214[0][0]                    
__________________________________________________________________________________________________
res3d_branch2a (Conv2D)         (None, 28, 28, 128)  65664       activation_656[0][0]             
__________________________________________________________________________________________________
bn3d_branch2a (BatchNormalizati (None, 28, 28, 128)  512         res3d_branch2a[0][0]             
__________________________________________________________________________________________________
activation_657 (Activation)     (None, 28, 28, 128)  0           bn3d_branch2a[0][0]              
__________________________________________________________________________________________________
res3d_branch2b (Conv2D)         (None, 28, 28, 128)  147584      activation_657[0][0]             
__________________________________________________________________________________________________
bn3d_branch2b (BatchNormalizati (None, 28, 28, 128)  512         res3d_branch2b[0][0]             
__________________________________________________________________________________________________
activation_658 (Activation)     (None, 28, 28, 128)  0           bn3d_branch2b[0][0]              
__________________________________________________________________________________________________
res3d_branch2c (Conv2D)         (None, 28, 28, 512)  66048       activation_658[0][0]             
__________________________________________________________________________________________________
bn3d_branch2c (BatchNormalizati (None, 28, 28, 512)  2048        res3d_branch2c[0][0]             
__________________________________________________________________________________________________
add_215 (Add)                   (None, 28, 28, 512)  0           bn3d_branch2c[0][0]              
                                                                 activation_656[0][0]             
__________________________________________________________________________________________________
activation_659 (Activation)     (None, 28, 28, 512)  0           add_215[0][0]                    
__________________________________________________________________________________________________
res4a_branch2a (Conv2D)         (None, 14, 14, 256)  131328      activation_659[0][0]             
__________________________________________________________________________________________________
bn4a_branch2a (BatchNormalizati (None, 14, 14, 256)  1024        res4a_branch2a[0][0]             
__________________________________________________________________________________________________
activation_660 (Activation)     (None, 14, 14, 256)  0           bn4a_branch2a[0][0]              
__________________________________________________________________________________________________
res4a_branch2b (Conv2D)         (None, 14, 14, 256)  590080      activation_660[0][0]             
__________________________________________________________________________________________________
bn4a_branch2b (BatchNormalizati (None, 14, 14, 256)  1024        res4a_branch2b[0][0]             
__________________________________________________________________________________________________
activation_661 (Activation)     (None, 14, 14, 256)  0           bn4a_branch2b[0][0]              
__________________________________________________________________________________________________
res4a_branch2c (Conv2D)         (None, 14, 14, 1024) 263168      activation_661[0][0]             
__________________________________________________________________________________________________
res4a_branch1 (Conv2D)          (None, 14, 14, 1024) 525312      activation_659[0][0]             
__________________________________________________________________________________________________
bn4a_branch2c (BatchNormalizati (None, 14, 14, 1024) 4096        res4a_branch2c[0][0]             
__________________________________________________________________________________________________
bn4a_branch1 (BatchNormalizatio (None, 14, 14, 1024) 4096        res4a_branch1[0][0]              
__________________________________________________________________________________________________
add_216 (Add)                   (None, 14, 14, 1024) 0           bn4a_branch2c[0][0]              
                                                                 bn4a_branch1[0][0]               
__________________________________________________________________________________________________
activation_662 (Activation)     (None, 14, 14, 1024) 0           add_216[0][0]                    
__________________________________________________________________________________________________
res4b_branch2a (Conv2D)         (None, 14, 14, 256)  262400      activation_662[0][0]             
__________________________________________________________________________________________________
bn4b_branch2a (BatchNormalizati (None, 14, 14, 256)  1024        res4b_branch2a[0][0]             
__________________________________________________________________________________________________
activation_663 (Activation)     (None, 14, 14, 256)  0           bn4b_branch2a[0][0]              
__________________________________________________________________________________________________
res4b_branch2b (Conv2D)         (None, 14, 14, 256)  590080      activation_663[0][0]             
__________________________________________________________________________________________________
bn4b_branch2b (BatchNormalizati (None, 14, 14, 256)  1024        res4b_branch2b[0][0]             
__________________________________________________________________________________________________
activation_664 (Activation)     (None, 14, 14, 256)  0           bn4b_branch2b[0][0]              
__________________________________________________________________________________________________
res4b_branch2c (Conv2D)         (None, 14, 14, 1024) 263168      activation_664[0][0]             
__________________________________________________________________________________________________
bn4b_branch2c (BatchNormalizati (None, 14, 14, 1024) 4096        res4b_branch2c[0][0]             
__________________________________________________________________________________________________
add_217 (Add)                   (None, 14, 14, 1024) 0           bn4b_branch2c[0][0]              
                                                                 activation_662[0][0]             
__________________________________________________________________________________________________
activation_665 (Activation)     (None, 14, 14, 1024) 0           add_217[0][0]                    
__________________________________________________________________________________________________
res4c_branch2a (Conv2D)         (None, 14, 14, 256)  262400      activation_665[0][0]             
__________________________________________________________________________________________________
bn4c_branch2a (BatchNormalizati (None, 14, 14, 256)  1024        res4c_branch2a[0][0]             
__________________________________________________________________________________________________
activation_666 (Activation)     (None, 14, 14, 256)  0           bn4c_branch2a[0][0]              
__________________________________________________________________________________________________
res4c_branch2b (Conv2D)         (None, 14, 14, 256)  590080      activation_666[0][0]             
__________________________________________________________________________________________________
bn4c_branch2b (BatchNormalizati (None, 14, 14, 256)  1024        res4c_branch2b[0][0]             
__________________________________________________________________________________________________
activation_667 (Activation)     (None, 14, 14, 256)  0           bn4c_branch2b[0][0]              
__________________________________________________________________________________________________
res4c_branch2c (Conv2D)         (None, 14, 14, 1024) 263168      activation_667[0][0]             
__________________________________________________________________________________________________
bn4c_branch2c (BatchNormalizati (None, 14, 14, 1024) 4096        res4c_branch2c[0][0]             
__________________________________________________________________________________________________
add_218 (Add)                   (None, 14, 14, 1024) 0           bn4c_branch2c[0][0]              
                                                                 activation_665[0][0]             
__________________________________________________________________________________________________
activation_668 (Activation)     (None, 14, 14, 1024) 0           add_218[0][0]                    
__________________________________________________________________________________________________
res4d_branch2a (Conv2D)         (None, 14, 14, 256)  262400      activation_668[0][0]             
__________________________________________________________________________________________________
bn4d_branch2a (BatchNormalizati (None, 14, 14, 256)  1024        res4d_branch2a[0][0]             
__________________________________________________________________________________________________
activation_669 (Activation)     (None, 14, 14, 256)  0           bn4d_branch2a[0][0]              
__________________________________________________________________________________________________
res4d_branch2b (Conv2D)         (None, 14, 14, 256)  590080      activation_669[0][0]             
__________________________________________________________________________________________________
bn4d_branch2b (BatchNormalizati (None, 14, 14, 256)  1024        res4d_branch2b[0][0]             
__________________________________________________________________________________________________
activation_670 (Activation)     (None, 14, 14, 256)  0           bn4d_branch2b[0][0]              
__________________________________________________________________________________________________
res4d_branch2c (Conv2D)         (None, 14, 14, 1024) 263168      activation_670[0][0]             
__________________________________________________________________________________________________
bn4d_branch2c (BatchNormalizati (None, 14, 14, 1024) 4096        res4d_branch2c[0][0]             
__________________________________________________________________________________________________
add_219 (Add)                   (None, 14, 14, 1024) 0           bn4d_branch2c[0][0]              
                                                                 activation_668[0][0]             
__________________________________________________________________________________________________
activation_671 (Activation)     (None, 14, 14, 1024) 0           add_219[0][0]                    
__________________________________________________________________________________________________
res4e_branch2a (Conv2D)         (None, 14, 14, 256)  262400      activation_671[0][0]             
__________________________________________________________________________________________________
bn4e_branch2a (BatchNormalizati (None, 14, 14, 256)  1024        res4e_branch2a[0][0]             
__________________________________________________________________________________________________
activation_672 (Activation)     (None, 14, 14, 256)  0           bn4e_branch2a[0][0]              
__________________________________________________________________________________________________
res4e_branch2b (Conv2D)         (None, 14, 14, 256)  590080      activation_672[0][0]             
__________________________________________________________________________________________________
bn4e_branch2b (BatchNormalizati (None, 14, 14, 256)  1024        res4e_branch2b[0][0]             
__________________________________________________________________________________________________
activation_673 (Activation)     (None, 14, 14, 256)  0           bn4e_branch2b[0][0]              
__________________________________________________________________________________________________
res4e_branch2c (Conv2D)         (None, 14, 14, 1024) 263168      activation_673[0][0]             
__________________________________________________________________________________________________
bn4e_branch2c (BatchNormalizati (None, 14, 14, 1024) 4096        res4e_branch2c[0][0]             
__________________________________________________________________________________________________
add_220 (Add)                   (None, 14, 14, 1024) 0           bn4e_branch2c[0][0]              
                                                                 activation_671[0][0]             
__________________________________________________________________________________________________
activation_674 (Activation)     (None, 14, 14, 1024) 0           add_220[0][0]                    
__________________________________________________________________________________________________
res4f_branch2a (Conv2D)         (None, 14, 14, 256)  262400      activation_674[0][0]             
__________________________________________________________________________________________________
bn4f_branch2a (BatchNormalizati (None, 14, 14, 256)  1024        res4f_branch2a[0][0]             
__________________________________________________________________________________________________
activation_675 (Activation)     (None, 14, 14, 256)  0           bn4f_branch2a[0][0]              
__________________________________________________________________________________________________
res4f_branch2b (Conv2D)         (None, 14, 14, 256)  590080      activation_675[0][0]             
__________________________________________________________________________________________________
bn4f_branch2b (BatchNormalizati (None, 14, 14, 256)  1024        res4f_branch2b[0][0]             
__________________________________________________________________________________________________
activation_676 (Activation)     (None, 14, 14, 256)  0           bn4f_branch2b[0][0]              
__________________________________________________________________________________________________
res4f_branch2c (Conv2D)         (None, 14, 14, 1024) 263168      activation_676[0][0]             
__________________________________________________________________________________________________
bn4f_branch2c (BatchNormalizati (None, 14, 14, 1024) 4096        res4f_branch2c[0][0]             
__________________________________________________________________________________________________
add_221 (Add)                   (None, 14, 14, 1024) 0           bn4f_branch2c[0][0]              
                                                                 activation_674[0][0]             
__________________________________________________________________________________________________
activation_677 (Activation)     (None, 14, 14, 1024) 0           add_221[0][0]                    
__________________________________________________________________________________________________
res5a_branch2a (Conv2D)         (None, 7, 7, 512)    524800      activation_677[0][0]             
__________________________________________________________________________________________________
bn5a_branch2a (BatchNormalizati (None, 7, 7, 512)    2048        res5a_branch2a[0][0]             
__________________________________________________________________________________________________
activation_678 (Activation)     (None, 7, 7, 512)    0           bn5a_branch2a[0][0]              
__________________________________________________________________________________________________
res5a_branch2b (Conv2D)         (None, 7, 7, 512)    2359808     activation_678[0][0]             
__________________________________________________________________________________________________
bn5a_branch2b (BatchNormalizati (None, 7, 7, 512)    2048        res5a_branch2b[0][0]             
__________________________________________________________________________________________________
activation_679 (Activation)     (None, 7, 7, 512)    0           bn5a_branch2b[0][0]              
__________________________________________________________________________________________________
res5a_branch2c (Conv2D)         (None, 7, 7, 2048)   1050624     activation_679[0][0]             
__________________________________________________________________________________________________
res5a_branch1 (Conv2D)          (None, 7, 7, 2048)   2099200     activation_677[0][0]             
__________________________________________________________________________________________________
bn5a_branch2c (BatchNormalizati (None, 7, 7, 2048)   8192        res5a_branch2c[0][0]             
__________________________________________________________________________________________________
bn5a_branch1 (BatchNormalizatio (None, 7, 7, 2048)   8192        res5a_branch1[0][0]              
__________________________________________________________________________________________________
add_222 (Add)                   (None, 7, 7, 2048)   0           bn5a_branch2c[0][0]              
                                                                 bn5a_branch1[0][0]               
__________________________________________________________________________________________________
activation_680 (Activation)     (None, 7, 7, 2048)   0           add_222[0][0]                    
__________________________________________________________________________________________________
res5b_branch2a (Conv2D)         (None, 7, 7, 512)    1049088     activation_680[0][0]             
__________________________________________________________________________________________________
bn5b_branch2a (BatchNormalizati (None, 7, 7, 512)    2048        res5b_branch2a[0][0]             
__________________________________________________________________________________________________
activation_681 (Activation)     (None, 7, 7, 512)    0           bn5b_branch2a[0][0]              
__________________________________________________________________________________________________
res5b_branch2b (Conv2D)         (None, 7, 7, 512)    2359808     activation_681[0][0]             
__________________________________________________________________________________________________
bn5b_branch2b (BatchNormalizati (None, 7, 7, 512)    2048        res5b_branch2b[0][0]             
__________________________________________________________________________________________________
activation_682 (Activation)     (None, 7, 7, 512)    0           bn5b_branch2b[0][0]              
__________________________________________________________________________________________________
res5b_branch2c (Conv2D)         (None, 7, 7, 2048)   1050624     activation_682[0][0]             
__________________________________________________________________________________________________
bn5b_branch2c (BatchNormalizati (None, 7, 7, 2048)   8192        res5b_branch2c[0][0]             
__________________________________________________________________________________________________
add_223 (Add)                   (None, 7, 7, 2048)   0           bn5b_branch2c[0][0]              
                                                                 activation_680[0][0]             
__________________________________________________________________________________________________
activation_683 (Activation)     (None, 7, 7, 2048)   0           add_223[0][0]                    
__________________________________________________________________________________________________
res5c_branch2a (Conv2D)         (None, 7, 7, 512)    1049088     activation_683[0][0]             
__________________________________________________________________________________________________
bn5c_branch2a (BatchNormalizati (None, 7, 7, 512)    2048        res5c_branch2a[0][0]             
__________________________________________________________________________________________________
activation_684 (Activation)     (None, 7, 7, 512)    0           bn5c_branch2a[0][0]              
__________________________________________________________________________________________________
res5c_branch2b (Conv2D)         (None, 7, 7, 512)    2359808     activation_684[0][0]             
__________________________________________________________________________________________________
bn5c_branch2b (BatchNormalizati (None, 7, 7, 512)    2048        res5c_branch2b[0][0]             
__________________________________________________________________________________________________
activation_685 (Activation)     (None, 7, 7, 512)    0           bn5c_branch2b[0][0]              
__________________________________________________________________________________________________
res5c_branch2c (Conv2D)         (None, 7, 7, 2048)   1050624     activation_685[0][0]             
__________________________________________________________________________________________________
bn5c_branch2c (BatchNormalizati (None, 7, 7, 2048)   8192        res5c_branch2c[0][0]             
__________________________________________________________________________________________________
add_224 (Add)                   (None, 7, 7, 2048)   0           bn5c_branch2c[0][0]              
                                                                 activation_683[0][0]             
__________________________________________________________________________________________________
activation_686 (Activation)     (None, 7, 7, 2048)   0           add_224[0][0]                    
__________________________________________________________________________________________________
global_average_pooling2d_13 (Gl (None, 2048)         0           activation_686[0][0]             
__________________________________________________________________________________________________
dense_25 (Dense)                (None, 4096)         8392704     global_average_pooling2d_13[0][0]
__________________________________________________________________________________________________
batch_normalization_13 (BatchNo (None, 4096)         16384       dense_25[0][0]                   
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 4096)         0           batch_normalization_13[0][0]     
__________________________________________________________________________________________________
dense_26 (Dense)                (None, 5004)         20501388    dropout_1[0][0]                  
==================================================================================================
Total params: 52,498,188
Trainable params: 28,955,404
Non-trainable params: 23,542,784
__________________________________________________________________________________________________
Epoch 1/50
474/474 [==============================] - 566s 1s/step - loss: 8.0635 - acc: 0.0465 - val_loss: 5.5734 - val_acc: 0.1665

Epoch 00001: val_acc improved from -inf to 0.16650, saving model to ./drive/My Drive/models/without_whale13/01-8.06-5.57.hdf5
Epoch 2/50
474/474 [==============================] - 508s 1s/step - loss: 5.7433 - acc: 0.1683 - val_loss: 3.6961 - val_acc: 0.3511

Epoch 00002: val_acc improved from 0.16650 to 0.35111, saving model to ./drive/My Drive/models/without_whale13/02-5.74-3.70.hdf5
Epoch 3/50
474/474 [==============================] - 507s 1s/step - loss: 3.5486 - acc: 0.3394 - val_loss: 3.6048 - val_acc: 0.3853

Epoch 00003: val_acc improved from 0.35111 to 0.38531, saving model to ./drive/My Drive/models/without_whale13/03-3.55-3.60.hdf5
Epoch 4/50
474/474 [==============================] - 507s 1s/step - loss: 1.9179 - acc: 0.5606 - val_loss: 3.1853 - val_acc: 0.4427

Epoch 00004: val_acc improved from 0.38531 to 0.44266, saving model to ./drive/My Drive/models/without_whale13/04-1.92-3.19.hdf5
Epoch 5/50
474/474 [==============================] - 506s 1s/step - loss: 1.0427 - acc: 0.7394 - val_loss: 3.2411 - val_acc: 0.4618

Epoch 00005: val_acc improved from 0.44266 to 0.46177, saving model to ./drive/My Drive/models/without_whale13/05-1.04-3.24.hdf5
Epoch 6/50
474/474 [==============================] - 506s 1s/step - loss: 0.7090 - acc: 0.8165 - val_loss: 3.5186 - val_acc: 0.4356

Epoch 00006: val_acc did not improve from 0.46177
Epoch 7/50
474/474 [==============================] - 506s 1s/step - loss: 0.5668 - acc: 0.8451 - val_loss: 3.2031 - val_acc: 0.5085

Epoch 00007: val_acc improved from 0.46177 to 0.50853, saving model to ./drive/My Drive/models/without_whale13/07-0.57-3.20.hdf5
Epoch 8/50
474/474 [==============================] - 507s 1s/step - loss: 0.4913 - acc: 0.8665 - val_loss: 2.9856 - val_acc: 0.5322

Epoch 00008: val_acc improved from 0.50853 to 0.53219, saving model to ./drive/My Drive/models/without_whale13/08-0.49-2.99.hdf5
Epoch 9/50
474/474 [==============================] - 506s 1s/step - loss: 0.4290 - acc: 0.8832 - val_loss: 3.2602 - val_acc: 0.5151

Epoch 00009: val_acc did not improve from 0.53219
Epoch 10/50
474/474 [==============================] - 508s 1s/step - loss: 0.3943 - acc: 0.8930 - val_loss: 3.1094 - val_acc: 0.5352

Epoch 00010: val_acc improved from 0.53219 to 0.53521, saving model to ./drive/My Drive/models/without_whale13/10-0.39-3.11.hdf5
Epoch 11/50
474/474 [==============================] - 509s 1s/step - loss: 0.3576 - acc: 0.9015 - val_loss: 3.2128 - val_acc: 0.5443

Epoch 00011: val_acc improved from 0.53521 to 0.54427, saving model to ./drive/My Drive/models/without_whale13/11-0.36-3.21.hdf5
Epoch 12/50
474/474 [==============================] - 510s 1s/step - loss: 0.3431 - acc: 0.9023 - val_loss: 3.2359 - val_acc: 0.5382

Epoch 00012: val_acc did not improve from 0.54427
Epoch 13/50
474/474 [==============================] - 509s 1s/step - loss: 0.2909 - acc: 0.9188 - val_loss: 3.0898 - val_acc: 0.5624

Epoch 00013: val_acc improved from 0.54427 to 0.56237, saving model to ./drive/My Drive/models/without_whale13/13-0.29-3.09.hdf5
Epoch 14/50
474/474 [==============================] - 507s 1s/step - loss: 0.2634 - acc: 0.9259 - val_loss: 3.2151 - val_acc: 0.5406

Epoch 00014: val_acc did not improve from 0.56237
Epoch 15/50
474/474 [==============================] - 504s 1s/step - loss: 0.2636 - acc: 0.9247 - val_loss: 3.1983 - val_acc: 0.5523

Epoch 00015: val_acc did not improve from 0.56237
Epoch 16/50
474/474 [==============================] - 503s 1s/step - loss: 0.2383 - acc: 0.9337 - val_loss: 3.2833 - val_acc: 0.5493

Epoch 00016: val_acc did not improve from 0.56237
Epoch 17/50
474/474 [==============================] - 505s 1s/step - loss: 0.2334 - acc: 0.9328 - val_loss: 3.3391 - val_acc: 0.5473

Epoch 00017: val_acc did not improve from 0.56237
Epoch 18/50
317/474 [===================>..........] - ETA: 2:40 - loss: 0.2201 - acc: 0.9392```



In [0]:
def get_model():
    conv_base = resnet50.ResNet50(weights='imagenet',
                      include_top=False,
                      input_shape=(input_size, input_size, 3))
    for layer in conv_base.layers[:]:
        if 'BatchNormalization' in str(layer):
            layer.trainable = True
        else:
            layer.trainable = False
    main_input = conv_base.input
    embedding = conv_base.output
    x = layers.GlobalAveragePooling2D()(embedding)
    x = layers.Dense(4096, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(5004, activation='softmax')(x)
    model = models.Model(inputs=[main_input], outputs=[x])
    model.compile(
        loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
    return model




```
Found 15160 images belonging to 5004 classes.
Found 537 images belonging to 5004 classes.
/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
==================================================================================================
input_15 (InputLayer)           (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_15[0][0]                   
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation_687 (Activation)     (None, 112, 112, 64) 0           bn_conv1[0][0]                   
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 114, 114, 64) 0           activation_687[0][0]             
__________________________________________________________________________________________________
max_pooling2d_15 (MaxPooling2D) (None, 56, 56, 64)   0           pool1_pad[0][0]                  
__________________________________________________________________________________________________
res2a_branch2a (Conv2D)         (None, 56, 56, 64)   4160        max_pooling2d_15[0][0]           
__________________________________________________________________________________________________
bn2a_branch2a (BatchNormalizati (None, 56, 56, 64)   256         res2a_branch2a[0][0]             
__________________________________________________________________________________________________
activation_688 (Activation)     (None, 56, 56, 64)   0           bn2a_branch2a[0][0]              
__________________________________________________________________________________________________
res2a_branch2b (Conv2D)         (None, 56, 56, 64)   36928       activation_688[0][0]             
__________________________________________________________________________________________________
bn2a_branch2b (BatchNormalizati (None, 56, 56, 64)   256         res2a_branch2b[0][0]             
__________________________________________________________________________________________________
activation_689 (Activation)     (None, 56, 56, 64)   0           bn2a_branch2b[0][0]              
__________________________________________________________________________________________________
res2a_branch2c (Conv2D)         (None, 56, 56, 256)  16640       activation_689[0][0]             
__________________________________________________________________________________________________
res2a_branch1 (Conv2D)          (None, 56, 56, 256)  16640       max_pooling2d_15[0][0]           
__________________________________________________________________________________________________
bn2a_branch2c (BatchNormalizati (None, 56, 56, 256)  1024        res2a_branch2c[0][0]             
__________________________________________________________________________________________________
bn2a_branch1 (BatchNormalizatio (None, 56, 56, 256)  1024        res2a_branch1[0][0]              
__________________________________________________________________________________________________
add_225 (Add)                   (None, 56, 56, 256)  0           bn2a_branch2c[0][0]              
                                                                 bn2a_branch1[0][0]               
__________________________________________________________________________________________________
activation_690 (Activation)     (None, 56, 56, 256)  0           add_225[0][0]                    
__________________________________________________________________________________________________
res2b_branch2a (Conv2D)         (None, 56, 56, 64)   16448       activation_690[0][0]             
__________________________________________________________________________________________________
bn2b_branch2a (BatchNormalizati (None, 56, 56, 64)   256         res2b_branch2a[0][0]             
__________________________________________________________________________________________________
activation_691 (Activation)     (None, 56, 56, 64)   0           bn2b_branch2a[0][0]              
__________________________________________________________________________________________________
res2b_branch2b (Conv2D)         (None, 56, 56, 64)   36928       activation_691[0][0]             
__________________________________________________________________________________________________
bn2b_branch2b (BatchNormalizati (None, 56, 56, 64)   256         res2b_branch2b[0][0]             
__________________________________________________________________________________________________
activation_692 (Activation)     (None, 56, 56, 64)   0           bn2b_branch2b[0][0]              
__________________________________________________________________________________________________
res2b_branch2c (Conv2D)         (None, 56, 56, 256)  16640       activation_692[0][0]             
__________________________________________________________________________________________________
bn2b_branch2c (BatchNormalizati (None, 56, 56, 256)  1024        res2b_branch2c[0][0]             
__________________________________________________________________________________________________
add_226 (Add)                   (None, 56, 56, 256)  0           bn2b_branch2c[0][0]              
                                                                 activation_690[0][0]             
__________________________________________________________________________________________________
activation_693 (Activation)     (None, 56, 56, 256)  0           add_226[0][0]                    
__________________________________________________________________________________________________
res2c_branch2a (Conv2D)         (None, 56, 56, 64)   16448       activation_693[0][0]             
__________________________________________________________________________________________________
bn2c_branch2a (BatchNormalizati (None, 56, 56, 64)   256         res2c_branch2a[0][0]             
__________________________________________________________________________________________________
activation_694 (Activation)     (None, 56, 56, 64)   0           bn2c_branch2a[0][0]              
__________________________________________________________________________________________________
res2c_branch2b (Conv2D)         (None, 56, 56, 64)   36928       activation_694[0][0]             
__________________________________________________________________________________________________
bn2c_branch2b (BatchNormalizati (None, 56, 56, 64)   256         res2c_branch2b[0][0]             
__________________________________________________________________________________________________
activation_695 (Activation)     (None, 56, 56, 64)   0           bn2c_branch2b[0][0]              
__________________________________________________________________________________________________
res2c_branch2c (Conv2D)         (None, 56, 56, 256)  16640       activation_695[0][0]             
__________________________________________________________________________________________________
bn2c_branch2c (BatchNormalizati (None, 56, 56, 256)  1024        res2c_branch2c[0][0]             
__________________________________________________________________________________________________
add_227 (Add)                   (None, 56, 56, 256)  0           bn2c_branch2c[0][0]              
                                                                 activation_693[0][0]             
__________________________________________________________________________________________________
activation_696 (Activation)     (None, 56, 56, 256)  0           add_227[0][0]                    
__________________________________________________________________________________________________
res3a_branch2a (Conv2D)         (None, 28, 28, 128)  32896       activation_696[0][0]             
__________________________________________________________________________________________________
bn3a_branch2a (BatchNormalizati (None, 28, 28, 128)  512         res3a_branch2a[0][0]             
__________________________________________________________________________________________________
activation_697 (Activation)     (None, 28, 28, 128)  0           bn3a_branch2a[0][0]              
__________________________________________________________________________________________________
res3a_branch2b (Conv2D)         (None, 28, 28, 128)  147584      activation_697[0][0]             
__________________________________________________________________________________________________
bn3a_branch2b (BatchNormalizati (None, 28, 28, 128)  512         res3a_branch2b[0][0]             
__________________________________________________________________________________________________
activation_698 (Activation)     (None, 28, 28, 128)  0           bn3a_branch2b[0][0]              
__________________________________________________________________________________________________
res3a_branch2c (Conv2D)         (None, 28, 28, 512)  66048       activation_698[0][0]             
__________________________________________________________________________________________________
res3a_branch1 (Conv2D)          (None, 28, 28, 512)  131584      activation_696[0][0]             
__________________________________________________________________________________________________
bn3a_branch2c (BatchNormalizati (None, 28, 28, 512)  2048        res3a_branch2c[0][0]             
__________________________________________________________________________________________________
bn3a_branch1 (BatchNormalizatio (None, 28, 28, 512)  2048        res3a_branch1[0][0]              
__________________________________________________________________________________________________
add_228 (Add)                   (None, 28, 28, 512)  0           bn3a_branch2c[0][0]              
                                                                 bn3a_branch1[0][0]               
__________________________________________________________________________________________________
activation_699 (Activation)     (None, 28, 28, 512)  0           add_228[0][0]                    
__________________________________________________________________________________________________
res3b_branch2a (Conv2D)         (None, 28, 28, 128)  65664       activation_699[0][0]             
__________________________________________________________________________________________________
bn3b_branch2a (BatchNormalizati (None, 28, 28, 128)  512         res3b_branch2a[0][0]             
__________________________________________________________________________________________________
activation_700 (Activation)     (None, 28, 28, 128)  0           bn3b_branch2a[0][0]              
__________________________________________________________________________________________________
res3b_branch2b (Conv2D)         (None, 28, 28, 128)  147584      activation_700[0][0]             
__________________________________________________________________________________________________
bn3b_branch2b (BatchNormalizati (None, 28, 28, 128)  512         res3b_branch2b[0][0]             
__________________________________________________________________________________________________
activation_701 (Activation)     (None, 28, 28, 128)  0           bn3b_branch2b[0][0]              
__________________________________________________________________________________________________
res3b_branch2c (Conv2D)         (None, 28, 28, 512)  66048       activation_701[0][0]             
__________________________________________________________________________________________________
bn3b_branch2c (BatchNormalizati (None, 28, 28, 512)  2048        res3b_branch2c[0][0]             
__________________________________________________________________________________________________
add_229 (Add)                   (None, 28, 28, 512)  0           bn3b_branch2c[0][0]              
                                                                 activation_699[0][0]             
__________________________________________________________________________________________________
activation_702 (Activation)     (None, 28, 28, 512)  0           add_229[0][0]                    
__________________________________________________________________________________________________
res3c_branch2a (Conv2D)         (None, 28, 28, 128)  65664       activation_702[0][0]             
__________________________________________________________________________________________________
bn3c_branch2a (BatchNormalizati (None, 28, 28, 128)  512         res3c_branch2a[0][0]             
__________________________________________________________________________________________________
activation_703 (Activation)     (None, 28, 28, 128)  0           bn3c_branch2a[0][0]              
__________________________________________________________________________________________________
res3c_branch2b (Conv2D)         (None, 28, 28, 128)  147584      activation_703[0][0]             
__________________________________________________________________________________________________
bn3c_branch2b (BatchNormalizati (None, 28, 28, 128)  512         res3c_branch2b[0][0]             
__________________________________________________________________________________________________
activation_704 (Activation)     (None, 28, 28, 128)  0           bn3c_branch2b[0][0]              
__________________________________________________________________________________________________
res3c_branch2c (Conv2D)         (None, 28, 28, 512)  66048       activation_704[0][0]             
__________________________________________________________________________________________________
bn3c_branch2c (BatchNormalizati (None, 28, 28, 512)  2048        res3c_branch2c[0][0]             
__________________________________________________________________________________________________
add_230 (Add)                   (None, 28, 28, 512)  0           bn3c_branch2c[0][0]              
                                                                 activation_702[0][0]             
__________________________________________________________________________________________________
activation_705 (Activation)     (None, 28, 28, 512)  0           add_230[0][0]                    
__________________________________________________________________________________________________
res3d_branch2a (Conv2D)         (None, 28, 28, 128)  65664       activation_705[0][0]             
__________________________________________________________________________________________________
bn3d_branch2a (BatchNormalizati (None, 28, 28, 128)  512         res3d_branch2a[0][0]             
__________________________________________________________________________________________________
activation_706 (Activation)     (None, 28, 28, 128)  0           bn3d_branch2a[0][0]              
__________________________________________________________________________________________________
res3d_branch2b (Conv2D)         (None, 28, 28, 128)  147584      activation_706[0][0]             
__________________________________________________________________________________________________
bn3d_branch2b (BatchNormalizati (None, 28, 28, 128)  512         res3d_branch2b[0][0]             
__________________________________________________________________________________________________
activation_707 (Activation)     (None, 28, 28, 128)  0           bn3d_branch2b[0][0]              
__________________________________________________________________________________________________
res3d_branch2c (Conv2D)         (None, 28, 28, 512)  66048       activation_707[0][0]             
__________________________________________________________________________________________________
bn3d_branch2c (BatchNormalizati (None, 28, 28, 512)  2048        res3d_branch2c[0][0]             
__________________________________________________________________________________________________
add_231 (Add)                   (None, 28, 28, 512)  0           bn3d_branch2c[0][0]              
                                                                 activation_705[0][0]             
__________________________________________________________________________________________________
activation_708 (Activation)     (None, 28, 28, 512)  0           add_231[0][0]                    
__________________________________________________________________________________________________
res4a_branch2a (Conv2D)         (None, 14, 14, 256)  131328      activation_708[0][0]             
__________________________________________________________________________________________________
bn4a_branch2a (BatchNormalizati (None, 14, 14, 256)  1024        res4a_branch2a[0][0]             
__________________________________________________________________________________________________
activation_709 (Activation)     (None, 14, 14, 256)  0           bn4a_branch2a[0][0]              
__________________________________________________________________________________________________
res4a_branch2b (Conv2D)         (None, 14, 14, 256)  590080      activation_709[0][0]             
__________________________________________________________________________________________________
bn4a_branch2b (BatchNormalizati (None, 14, 14, 256)  1024        res4a_branch2b[0][0]             
__________________________________________________________________________________________________
activation_710 (Activation)     (None, 14, 14, 256)  0           bn4a_branch2b[0][0]              
__________________________________________________________________________________________________
res4a_branch2c (Conv2D)         (None, 14, 14, 1024) 263168      activation_710[0][0]             
__________________________________________________________________________________________________
res4a_branch1 (Conv2D)          (None, 14, 14, 1024) 525312      activation_708[0][0]             
__________________________________________________________________________________________________
bn4a_branch2c (BatchNormalizati (None, 14, 14, 1024) 4096        res4a_branch2c[0][0]             
__________________________________________________________________________________________________
bn4a_branch1 (BatchNormalizatio (None, 14, 14, 1024) 4096        res4a_branch1[0][0]              
__________________________________________________________________________________________________
add_232 (Add)                   (None, 14, 14, 1024) 0           bn4a_branch2c[0][0]              
                                                                 bn4a_branch1[0][0]               
__________________________________________________________________________________________________
activation_711 (Activation)     (None, 14, 14, 1024) 0           add_232[0][0]                    
__________________________________________________________________________________________________
res4b_branch2a (Conv2D)         (None, 14, 14, 256)  262400      activation_711[0][0]             
__________________________________________________________________________________________________
bn4b_branch2a (BatchNormalizati (None, 14, 14, 256)  1024        res4b_branch2a[0][0]             
__________________________________________________________________________________________________
activation_712 (Activation)     (None, 14, 14, 256)  0           bn4b_branch2a[0][0]              
__________________________________________________________________________________________________
res4b_branch2b (Conv2D)         (None, 14, 14, 256)  590080      activation_712[0][0]             
__________________________________________________________________________________________________
bn4b_branch2b (BatchNormalizati (None, 14, 14, 256)  1024        res4b_branch2b[0][0]             
__________________________________________________________________________________________________
activation_713 (Activation)     (None, 14, 14, 256)  0           bn4b_branch2b[0][0]              
__________________________________________________________________________________________________
res4b_branch2c (Conv2D)         (None, 14, 14, 1024) 263168      activation_713[0][0]             
__________________________________________________________________________________________________
bn4b_branch2c (BatchNormalizati (None, 14, 14, 1024) 4096        res4b_branch2c[0][0]             
__________________________________________________________________________________________________
add_233 (Add)                   (None, 14, 14, 1024) 0           bn4b_branch2c[0][0]              
                                                                 activation_711[0][0]             
__________________________________________________________________________________________________
activation_714 (Activation)     (None, 14, 14, 1024) 0           add_233[0][0]                    
__________________________________________________________________________________________________
res4c_branch2a (Conv2D)         (None, 14, 14, 256)  262400      activation_714[0][0]             
__________________________________________________________________________________________________
bn4c_branch2a (BatchNormalizati (None, 14, 14, 256)  1024        res4c_branch2a[0][0]             
__________________________________________________________________________________________________
activation_715 (Activation)     (None, 14, 14, 256)  0           bn4c_branch2a[0][0]              
__________________________________________________________________________________________________
res4c_branch2b (Conv2D)         (None, 14, 14, 256)  590080      activation_715[0][0]             
__________________________________________________________________________________________________
bn4c_branch2b (BatchNormalizati (None, 14, 14, 256)  1024        res4c_branch2b[0][0]             
__________________________________________________________________________________________________
activation_716 (Activation)     (None, 14, 14, 256)  0           bn4c_branch2b[0][0]              
__________________________________________________________________________________________________
res4c_branch2c (Conv2D)         (None, 14, 14, 1024) 263168      activation_716[0][0]             
__________________________________________________________________________________________________
bn4c_branch2c (BatchNormalizati (None, 14, 14, 1024) 4096        res4c_branch2c[0][0]             
__________________________________________________________________________________________________
add_234 (Add)                   (None, 14, 14, 1024) 0           bn4c_branch2c[0][0]              
                                                                 activation_714[0][0]             
__________________________________________________________________________________________________
activation_717 (Activation)     (None, 14, 14, 1024) 0           add_234[0][0]                    
__________________________________________________________________________________________________
res4d_branch2a (Conv2D)         (None, 14, 14, 256)  262400      activation_717[0][0]             
__________________________________________________________________________________________________
bn4d_branch2a (BatchNormalizati (None, 14, 14, 256)  1024        res4d_branch2a[0][0]             
__________________________________________________________________________________________________
activation_718 (Activation)     (None, 14, 14, 256)  0           bn4d_branch2a[0][0]              
__________________________________________________________________________________________________
res4d_branch2b (Conv2D)         (None, 14, 14, 256)  590080      activation_718[0][0]             
__________________________________________________________________________________________________
bn4d_branch2b (BatchNormalizati (None, 14, 14, 256)  1024        res4d_branch2b[0][0]             
__________________________________________________________________________________________________
activation_719 (Activation)     (None, 14, 14, 256)  0           bn4d_branch2b[0][0]              
__________________________________________________________________________________________________
res4d_branch2c (Conv2D)         (None, 14, 14, 1024) 263168      activation_719[0][0]             
__________________________________________________________________________________________________
bn4d_branch2c (BatchNormalizati (None, 14, 14, 1024) 4096        res4d_branch2c[0][0]             
__________________________________________________________________________________________________
add_235 (Add)                   (None, 14, 14, 1024) 0           bn4d_branch2c[0][0]              
                                                                 activation_717[0][0]             
__________________________________________________________________________________________________
activation_720 (Activation)     (None, 14, 14, 1024) 0           add_235[0][0]                    
__________________________________________________________________________________________________
res4e_branch2a (Conv2D)         (None, 14, 14, 256)  262400      activation_720[0][0]             
__________________________________________________________________________________________________
bn4e_branch2a (BatchNormalizati (None, 14, 14, 256)  1024        res4e_branch2a[0][0]             
__________________________________________________________________________________________________
activation_721 (Activation)     (None, 14, 14, 256)  0           bn4e_branch2a[0][0]              
__________________________________________________________________________________________________
res4e_branch2b (Conv2D)         (None, 14, 14, 256)  590080      activation_721[0][0]             
__________________________________________________________________________________________________
bn4e_branch2b (BatchNormalizati (None, 14, 14, 256)  1024        res4e_branch2b[0][0]             
__________________________________________________________________________________________________
activation_722 (Activation)     (None, 14, 14, 256)  0           bn4e_branch2b[0][0]              
__________________________________________________________________________________________________
res4e_branch2c (Conv2D)         (None, 14, 14, 1024) 263168      activation_722[0][0]             
__________________________________________________________________________________________________
bn4e_branch2c (BatchNormalizati (None, 14, 14, 1024) 4096        res4e_branch2c[0][0]             
__________________________________________________________________________________________________
add_236 (Add)                   (None, 14, 14, 1024) 0           bn4e_branch2c[0][0]              
                                                                 activation_720[0][0]             
__________________________________________________________________________________________________
activation_723 (Activation)     (None, 14, 14, 1024) 0           add_236[0][0]                    
__________________________________________________________________________________________________
res4f_branch2a (Conv2D)         (None, 14, 14, 256)  262400      activation_723[0][0]             
__________________________________________________________________________________________________
bn4f_branch2a (BatchNormalizati (None, 14, 14, 256)  1024        res4f_branch2a[0][0]             
__________________________________________________________________________________________________
activation_724 (Activation)     (None, 14, 14, 256)  0           bn4f_branch2a[0][0]              
__________________________________________________________________________________________________
res4f_branch2b (Conv2D)         (None, 14, 14, 256)  590080      activation_724[0][0]             
__________________________________________________________________________________________________
bn4f_branch2b (BatchNormalizati (None, 14, 14, 256)  1024        res4f_branch2b[0][0]             
__________________________________________________________________________________________________
activation_725 (Activation)     (None, 14, 14, 256)  0           bn4f_branch2b[0][0]              
__________________________________________________________________________________________________
res4f_branch2c (Conv2D)         (None, 14, 14, 1024) 263168      activation_725[0][0]             
__________________________________________________________________________________________________
bn4f_branch2c (BatchNormalizati (None, 14, 14, 1024) 4096        res4f_branch2c[0][0]             
__________________________________________________________________________________________________
add_237 (Add)                   (None, 14, 14, 1024) 0           bn4f_branch2c[0][0]              
                                                                 activation_723[0][0]             
__________________________________________________________________________________________________
activation_726 (Activation)     (None, 14, 14, 1024) 0           add_237[0][0]                    
__________________________________________________________________________________________________
res5a_branch2a (Conv2D)         (None, 7, 7, 512)    524800      activation_726[0][0]             
__________________________________________________________________________________________________
bn5a_branch2a (BatchNormalizati (None, 7, 7, 512)    2048        res5a_branch2a[0][0]             
__________________________________________________________________________________________________
activation_727 (Activation)     (None, 7, 7, 512)    0           bn5a_branch2a[0][0]              
__________________________________________________________________________________________________
res5a_branch2b (Conv2D)         (None, 7, 7, 512)    2359808     activation_727[0][0]             
__________________________________________________________________________________________________
bn5a_branch2b (BatchNormalizati (None, 7, 7, 512)    2048        res5a_branch2b[0][0]             
__________________________________________________________________________________________________
activation_728 (Activation)     (None, 7, 7, 512)    0           bn5a_branch2b[0][0]              
__________________________________________________________________________________________________
res5a_branch2c (Conv2D)         (None, 7, 7, 2048)   1050624     activation_728[0][0]             
__________________________________________________________________________________________________
res5a_branch1 (Conv2D)          (None, 7, 7, 2048)   2099200     activation_726[0][0]             
__________________________________________________________________________________________________
bn5a_branch2c (BatchNormalizati (None, 7, 7, 2048)   8192        res5a_branch2c[0][0]             
__________________________________________________________________________________________________
bn5a_branch1 (BatchNormalizatio (None, 7, 7, 2048)   8192        res5a_branch1[0][0]              
__________________________________________________________________________________________________
add_238 (Add)                   (None, 7, 7, 2048)   0           bn5a_branch2c[0][0]              
                                                                 bn5a_branch1[0][0]               
__________________________________________________________________________________________________
activation_729 (Activation)     (None, 7, 7, 2048)   0           add_238[0][0]                    
__________________________________________________________________________________________________
res5b_branch2a (Conv2D)         (None, 7, 7, 512)    1049088     activation_729[0][0]             
__________________________________________________________________________________________________
bn5b_branch2a (BatchNormalizati (None, 7, 7, 512)    2048        res5b_branch2a[0][0]             
__________________________________________________________________________________________________
activation_730 (Activation)     (None, 7, 7, 512)    0           bn5b_branch2a[0][0]              
__________________________________________________________________________________________________
res5b_branch2b (Conv2D)         (None, 7, 7, 512)    2359808     activation_730[0][0]             
__________________________________________________________________________________________________
bn5b_branch2b (BatchNormalizati (None, 7, 7, 512)    2048        res5b_branch2b[0][0]             
__________________________________________________________________________________________________
activation_731 (Activation)     (None, 7, 7, 512)    0           bn5b_branch2b[0][0]              
__________________________________________________________________________________________________
res5b_branch2c (Conv2D)         (None, 7, 7, 2048)   1050624     activation_731[0][0]             
__________________________________________________________________________________________________
bn5b_branch2c (BatchNormalizati (None, 7, 7, 2048)   8192        res5b_branch2c[0][0]             
__________________________________________________________________________________________________
add_239 (Add)                   (None, 7, 7, 2048)   0           bn5b_branch2c[0][0]              
                                                                 activation_729[0][0]             
__________________________________________________________________________________________________
activation_732 (Activation)     (None, 7, 7, 2048)   0           add_239[0][0]                    
__________________________________________________________________________________________________
res5c_branch2a (Conv2D)         (None, 7, 7, 512)    1049088     activation_732[0][0]             
__________________________________________________________________________________________________
bn5c_branch2a (BatchNormalizati (None, 7, 7, 512)    2048        res5c_branch2a[0][0]             
__________________________________________________________________________________________________
activation_733 (Activation)     (None, 7, 7, 512)    0           bn5c_branch2a[0][0]              
__________________________________________________________________________________________________
res5c_branch2b (Conv2D)         (None, 7, 7, 512)    2359808     activation_733[0][0]             
__________________________________________________________________________________________________
bn5c_branch2b (BatchNormalizati (None, 7, 7, 512)    2048        res5c_branch2b[0][0]             
__________________________________________________________________________________________________
activation_734 (Activation)     (None, 7, 7, 512)    0           bn5c_branch2b[0][0]              
__________________________________________________________________________________________________
res5c_branch2c (Conv2D)         (None, 7, 7, 2048)   1050624     activation_734[0][0]             
__________________________________________________________________________________________________
bn5c_branch2c (BatchNormalizati (None, 7, 7, 2048)   8192        res5c_branch2c[0][0]             
__________________________________________________________________________________________________
add_240 (Add)                   (None, 7, 7, 2048)   0           bn5c_branch2c[0][0]              
                                                                 activation_732[0][0]             
__________________________________________________________________________________________________
activation_735 (Activation)     (None, 7, 7, 2048)   0           add_240[0][0]                    
__________________________________________________________________________________________________
global_average_pooling2d_14 (Gl (None, 2048)         0           activation_735[0][0]             
__________________________________________________________________________________________________
dense_27 (Dense)                (None, 4096)         8392704     global_average_pooling2d_14[0][0]
__________________________________________________________________________________________________
batch_normalization_14 (BatchNo (None, 4096)         16384       dense_27[0][0]                   
__________________________________________________________________________________________________
dropout_2 (Dropout)             (None, 4096)         0           batch_normalization_14[0][0]     
__________________________________________________________________________________________________
dense_28 (Dense)                (None, 5004)         20501388    dropout_2[0][0]                  
==================================================================================================
Total params: 52,498,188
Trainable params: 28,955,404
Non-trainable params: 23,542,784
__________________________________________________________________________________________________
Epoch 1/50
474/474 [==============================] - 579s 1s/step - loss: 8.2339 - acc: 0.0393 - val_loss: 5.5208 - val_acc: 0.1735

Epoch 00001: val_acc improved from -inf to 0.17352, saving model to ./drive/My Drive/models/without_whale13/01-8.23-5.52.hdf5
Epoch 2/50
474/474 [==============================] - 509s 1s/step - loss: 6.6319 - acc: 0.1238 - val_loss: 4.0748 - val_acc: 0.3089

Epoch 00002: val_acc improved from 0.17352 to 0.30885, saving model to ./drive/My Drive/models/without_whale13/02-6.63-4.07.hdf5
Epoch 3/50
474/474 [==============================] - 507s 1s/step - loss: 4.8483 - acc: 0.2322 - val_loss: 3.3734 - val_acc: 0.3954

Epoch 00003: val_acc improved from 0.30885 to 0.39537, saving model to ./drive/My Drive/models/without_whale13/03-4.85-3.37.hdf5
Epoch 4/50
474/474 [==============================] - 512s 1s/step - loss: 3.3177 - acc: 0.3772 - val_loss: 2.9827 - val_acc: 0.4306

Epoch 00004: val_acc improved from 0.39537 to 0.43058, saving model to ./drive/My Drive/models/without_whale13/04-3.32-2.98.hdf5
Epoch 5/50
474/474 [==============================] - 512s 1s/step - loss: 2.1117 - acc: 0.5268 - val_loss: 3.1498 - val_acc: 0.4829

Epoch 00005: val_acc improved from 0.43058 to 0.48290, saving model to ./drive/My Drive/models/without_whale13/05-2.11-3.15.hdf5
Epoch 6/50
474/474 [==============================] - 512s 1s/step - loss: 1.3329 - acc: 0.6797 - val_loss: 3.0583 - val_acc: 0.4930

Epoch 00006: val_acc improved from 0.48290 to 0.49296, saving model to ./drive/My Drive/models/without_whale13/06-1.33-3.06.hdf5
Epoch 7/50
474/474 [==============================] - 508s 1s/step - loss: 0.9801 - acc: 0.7529 - val_loss: 3.1736 - val_acc: 0.4985

Epoch 00007: val_acc improved from 0.49296 to 0.49850, saving model to ./drive/My Drive/models/without_whale13/07-0.98-3.17.hdf5
Epoch 8/50
474/474 [==============================] - 507s 1s/step - loss: 0.7370 - acc: 0.8056 - val_loss: 2.9600 - val_acc: 0.5181

Epoch 00008: val_acc improved from 0.49850 to 0.51811, saving model to ./drive/My Drive/models/without_whale13/08-0.74-2.96.hdf5
Epoch 9/50
474/474 [==============================] - 506s 1s/step - loss: 0.6368 - acc: 0.8314 - val_loss: 3.3870 - val_acc: 0.5030

Epoch 00009: val_acc did not improve from 0.51811
Epoch 10/50
474/474 [==============================] - 503s 1s/step - loss: 0.5866 - acc: 0.8421 - val_loss: 3.0493 - val_acc: 0.5111

Epoch 00010: val_acc did not improve from 0.51811
Epoch 11/50
474/474 [==============================] - 506s 1s/step - loss: 0.5431 - acc: 0.8547 - val_loss: 3.5011 - val_acc: 0.5010

Epoch 00011: val_acc did not improve from 0.51811
Epoch 12/50
 97/474 [=====>........................] - ETA: 6:07 - loss: 0.3649 - acc: 0.9030```



In [0]:
train_generator, val_generator = load_data()
model_wrapper = ModelWrapper()
model = model_wrapper.get_model()
model.summary()

model_dir = os.path.join(model_base_dir, 'without_whale' + model_wrapper.name)
model_dir = os.path.join('./drive/My Drive', model_dir)

os.makedirs(model_dir, exist_ok=True)

model_checkpoint_path = os.path.join(model_dir, '{epoch:02d}-{loss:.2f}-{val_loss:.2f}.hdf5')
model_checkpoint = ModelCheckpoint(model_checkpoint_path, monitor='val_acc', verbose=1, save_best_only=True)
csv_log_path = os.path.join(model_dir, 'train.log')
csv_logger = CSVLogger(csv_log_path)
history = model.fit_generator(train_generator,
                    steps_per_epoch=len(train_generator),
                    epochs=epochs,
                    validation_data=val_generator,
                    validation_steps=50,
                    callbacks=[model_checkpoint, csv_logger]
                    )

new model trainers

In [0]:
class ModelWrapper():
    def __init__(self):
        self.name = 'v5'
    def get_model(self):
        conv_base = resnet50.ResNet50(weights='imagenet',
                                      include_top=False,
                                      input_shape=(input_size, input_size, 3))
        for layer in conv_base.layers[:]:
            if 'BatchNormalization' in str(layer):
                layer.trainable = True
            else:
                layer.trainable = False
        main_input = conv_base.input
        embedding = conv_base.output
        x = layers.GlobalAveragePooling2D()(embedding)
        x = layers.Dropout(0.5)(x)
        x = layers.Dense(4096, activation='relu')(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.5)(x)
        x = layers.Dense(5004, activation='softmax')(x)
        model = models.Model(inputs=[main_input], outputs=[x])
        model.compile(
            loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
        return model


In [0]:
from keras import regularizers
class ModelV6():
    def __init__(self):
        self.name = 'v6'
    def get_model(self):
        conv_base = resnet50.ResNet50(weights='imagenet',
                                      include_top=False,
                                      input_shape=(input_size, input_size, 3))
        for layer in conv_base.layers[:]:
            if 'BatchNormalization' in str(layer):
                layer.trainable = True
            else:
                layer.trainable = False
        main_input = conv_base.input
        embedding = conv_base.output
        x = layers.GlobalAveragePooling2D()(embedding)
        x = layers.Dropout(0.5)(x)
        x = layers.Dense(4096, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.5)(x)
        x = layers.Dense(5004, activation='softmax')(x)
        model = models.Model(inputs=[main_input], outputs=[x])
        model.compile(
            loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
        return model


 

```
Found 15160 images belonging to 5004 classes.
Found 537 images belonging to 5004 classes.
/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
==================================================================================================
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation_50 (Activation)      (None, 112, 112, 64) 0           bn_conv1[0][0]                   
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 114, 114, 64) 0           activation_50[0][0]              
__________________________________________________________________________________________________
max_pooling2d_2 (MaxPooling2D)  (None, 56, 56, 64)   0           pool1_pad[0][0]                  
__________________________________________________________________________________________________
res2a_branch2a (Conv2D)         (None, 56, 56, 64)   4160        max_pooling2d_2[0][0]            
__________________________________________________________________________________________________
bn2a_branch2a (BatchNormalizati (None, 56, 56, 64)   256         res2a_branch2a[0][0]             
__________________________________________________________________________________________________
activation_51 (Activation)      (None, 56, 56, 64)   0           bn2a_branch2a[0][0]              
__________________________________________________________________________________________________
res2a_branch2b (Conv2D)         (None, 56, 56, 64)   36928       activation_51[0][0]              
__________________________________________________________________________________________________
bn2a_branch2b (BatchNormalizati (None, 56, 56, 64)   256         res2a_branch2b[0][0]             
__________________________________________________________________________________________________
activation_52 (Activation)      (None, 56, 56, 64)   0           bn2a_branch2b[0][0]              
__________________________________________________________________________________________________
res2a_branch2c (Conv2D)         (None, 56, 56, 256)  16640       activation_52[0][0]              
__________________________________________________________________________________________________
res2a_branch1 (Conv2D)          (None, 56, 56, 256)  16640       max_pooling2d_2[0][0]            
__________________________________________________________________________________________________
bn2a_branch2c (BatchNormalizati (None, 56, 56, 256)  1024        res2a_branch2c[0][0]             
__________________________________________________________________________________________________
bn2a_branch1 (BatchNormalizatio (None, 56, 56, 256)  1024        res2a_branch1[0][0]              
__________________________________________________________________________________________________
add_17 (Add)                    (None, 56, 56, 256)  0           bn2a_branch2c[0][0]              
                                                                 bn2a_branch1[0][0]               
__________________________________________________________________________________________________
activation_53 (Activation)      (None, 56, 56, 256)  0           add_17[0][0]                     
__________________________________________________________________________________________________
res2b_branch2a (Conv2D)         (None, 56, 56, 64)   16448       activation_53[0][0]              
__________________________________________________________________________________________________
bn2b_branch2a (BatchNormalizati (None, 56, 56, 64)   256         res2b_branch2a[0][0]             
__________________________________________________________________________________________________
activation_54 (Activation)      (None, 56, 56, 64)   0           bn2b_branch2a[0][0]              
__________________________________________________________________________________________________
res2b_branch2b (Conv2D)         (None, 56, 56, 64)   36928       activation_54[0][0]              
__________________________________________________________________________________________________
bn2b_branch2b (BatchNormalizati (None, 56, 56, 64)   256         res2b_branch2b[0][0]             
__________________________________________________________________________________________________
activation_55 (Activation)      (None, 56, 56, 64)   0           bn2b_branch2b[0][0]              
__________________________________________________________________________________________________
res2b_branch2c (Conv2D)         (None, 56, 56, 256)  16640       activation_55[0][0]              
__________________________________________________________________________________________________
bn2b_branch2c (BatchNormalizati (None, 56, 56, 256)  1024        res2b_branch2c[0][0]             
__________________________________________________________________________________________________
add_18 (Add)                    (None, 56, 56, 256)  0           bn2b_branch2c[0][0]              
                                                                 activation_53[0][0]              
__________________________________________________________________________________________________
activation_56 (Activation)      (None, 56, 56, 256)  0           add_18[0][0]                     
__________________________________________________________________________________________________
res2c_branch2a (Conv2D)         (None, 56, 56, 64)   16448       activation_56[0][0]              
__________________________________________________________________________________________________
bn2c_branch2a (BatchNormalizati (None, 56, 56, 64)   256         res2c_branch2a[0][0]             
__________________________________________________________________________________________________
activation_57 (Activation)      (None, 56, 56, 64)   0           bn2c_branch2a[0][0]              
__________________________________________________________________________________________________
res2c_branch2b (Conv2D)         (None, 56, 56, 64)   36928       activation_57[0][0]              
__________________________________________________________________________________________________
bn2c_branch2b (BatchNormalizati (None, 56, 56, 64)   256         res2c_branch2b[0][0]             
__________________________________________________________________________________________________
activation_58 (Activation)      (None, 56, 56, 64)   0           bn2c_branch2b[0][0]              
__________________________________________________________________________________________________
res2c_branch2c (Conv2D)         (None, 56, 56, 256)  16640       activation_58[0][0]              
__________________________________________________________________________________________________
bn2c_branch2c (BatchNormalizati (None, 56, 56, 256)  1024        res2c_branch2c[0][0]             
__________________________________________________________________________________________________
add_19 (Add)                    (None, 56, 56, 256)  0           bn2c_branch2c[0][0]              
                                                                 activation_56[0][0]              
__________________________________________________________________________________________________
activation_59 (Activation)      (None, 56, 56, 256)  0           add_19[0][0]                     
__________________________________________________________________________________________________
res3a_branch2a (Conv2D)         (None, 28, 28, 128)  32896       activation_59[0][0]              
__________________________________________________________________________________________________
bn3a_branch2a (BatchNormalizati (None, 28, 28, 128)  512         res3a_branch2a[0][0]             
__________________________________________________________________________________________________
activation_60 (Activation)      (None, 28, 28, 128)  0           bn3a_branch2a[0][0]              
__________________________________________________________________________________________________
res3a_branch2b (Conv2D)         (None, 28, 28, 128)  147584      activation_60[0][0]              
__________________________________________________________________________________________________
bn3a_branch2b (BatchNormalizati (None, 28, 28, 128)  512         res3a_branch2b[0][0]             
__________________________________________________________________________________________________
activation_61 (Activation)      (None, 28, 28, 128)  0           bn3a_branch2b[0][0]              
__________________________________________________________________________________________________
res3a_branch2c (Conv2D)         (None, 28, 28, 512)  66048       activation_61[0][0]              
__________________________________________________________________________________________________
res3a_branch1 (Conv2D)          (None, 28, 28, 512)  131584      activation_59[0][0]              
__________________________________________________________________________________________________
bn3a_branch2c (BatchNormalizati (None, 28, 28, 512)  2048        res3a_branch2c[0][0]             
__________________________________________________________________________________________________
bn3a_branch1 (BatchNormalizatio (None, 28, 28, 512)  2048        res3a_branch1[0][0]              
__________________________________________________________________________________________________
add_20 (Add)                    (None, 28, 28, 512)  0           bn3a_branch2c[0][0]              
                                                                 bn3a_branch1[0][0]               
__________________________________________________________________________________________________
activation_62 (Activation)      (None, 28, 28, 512)  0           add_20[0][0]                     
__________________________________________________________________________________________________
res3b_branch2a (Conv2D)         (None, 28, 28, 128)  65664       activation_62[0][0]              
__________________________________________________________________________________________________
bn3b_branch2a (BatchNormalizati (None, 28, 28, 128)  512         res3b_branch2a[0][0]             
__________________________________________________________________________________________________
activation_63 (Activation)      (None, 28, 28, 128)  0           bn3b_branch2a[0][0]              
__________________________________________________________________________________________________
res3b_branch2b (Conv2D)         (None, 28, 28, 128)  147584      activation_63[0][0]              
__________________________________________________________________________________________________
bn3b_branch2b (BatchNormalizati (None, 28, 28, 128)  512         res3b_branch2b[0][0]             
__________________________________________________________________________________________________
activation_64 (Activation)      (None, 28, 28, 128)  0           bn3b_branch2b[0][0]              
__________________________________________________________________________________________________
res3b_branch2c (Conv2D)         (None, 28, 28, 512)  66048       activation_64[0][0]              
__________________________________________________________________________________________________
bn3b_branch2c (BatchNormalizati (None, 28, 28, 512)  2048        res3b_branch2c[0][0]             
__________________________________________________________________________________________________
add_21 (Add)                    (None, 28, 28, 512)  0           bn3b_branch2c[0][0]              
                                                                 activation_62[0][0]              
__________________________________________________________________________________________________
activation_65 (Activation)      (None, 28, 28, 512)  0           add_21[0][0]                     
__________________________________________________________________________________________________
res3c_branch2a (Conv2D)         (None, 28, 28, 128)  65664       activation_65[0][0]              
__________________________________________________________________________________________________
bn3c_branch2a (BatchNormalizati (None, 28, 28, 128)  512         res3c_branch2a[0][0]             
__________________________________________________________________________________________________
activation_66 (Activation)      (None, 28, 28, 128)  0           bn3c_branch2a[0][0]              
__________________________________________________________________________________________________
res3c_branch2b (Conv2D)         (None, 28, 28, 128)  147584      activation_66[0][0]              
__________________________________________________________________________________________________
bn3c_branch2b (BatchNormalizati (None, 28, 28, 128)  512         res3c_branch2b[0][0]             
__________________________________________________________________________________________________
activation_67 (Activation)      (None, 28, 28, 128)  0           bn3c_branch2b[0][0]              
__________________________________________________________________________________________________
res3c_branch2c (Conv2D)         (None, 28, 28, 512)  66048       activation_67[0][0]              
__________________________________________________________________________________________________
bn3c_branch2c (BatchNormalizati (None, 28, 28, 512)  2048        res3c_branch2c[0][0]             
__________________________________________________________________________________________________
add_22 (Add)                    (None, 28, 28, 512)  0           bn3c_branch2c[0][0]              
                                                                 activation_65[0][0]              
__________________________________________________________________________________________________
activation_68 (Activation)      (None, 28, 28, 512)  0           add_22[0][0]                     
__________________________________________________________________________________________________
res3d_branch2a (Conv2D)         (None, 28, 28, 128)  65664       activation_68[0][0]              
__________________________________________________________________________________________________
bn3d_branch2a (BatchNormalizati (None, 28, 28, 128)  512         res3d_branch2a[0][0]             
__________________________________________________________________________________________________
activation_69 (Activation)      (None, 28, 28, 128)  0           bn3d_branch2a[0][0]              
__________________________________________________________________________________________________
res3d_branch2b (Conv2D)         (None, 28, 28, 128)  147584      activation_69[0][0]              
__________________________________________________________________________________________________
bn3d_branch2b (BatchNormalizati (None, 28, 28, 128)  512         res3d_branch2b[0][0]             
__________________________________________________________________________________________________
activation_70 (Activation)      (None, 28, 28, 128)  0           bn3d_branch2b[0][0]              
__________________________________________________________________________________________________
res3d_branch2c (Conv2D)         (None, 28, 28, 512)  66048       activation_70[0][0]              
__________________________________________________________________________________________________
bn3d_branch2c (BatchNormalizati (None, 28, 28, 512)  2048        res3d_branch2c[0][0]             
__________________________________________________________________________________________________
add_23 (Add)                    (None, 28, 28, 512)  0           bn3d_branch2c[0][0]              
                                                                 activation_68[0][0]              
__________________________________________________________________________________________________
activation_71 (Activation)      (None, 28, 28, 512)  0           add_23[0][0]                     
__________________________________________________________________________________________________
res4a_branch2a (Conv2D)         (None, 14, 14, 256)  131328      activation_71[0][0]              
__________________________________________________________________________________________________
bn4a_branch2a (BatchNormalizati (None, 14, 14, 256)  1024        res4a_branch2a[0][0]             
__________________________________________________________________________________________________
activation_72 (Activation)      (None, 14, 14, 256)  0           bn4a_branch2a[0][0]              
__________________________________________________________________________________________________
res4a_branch2b (Conv2D)         (None, 14, 14, 256)  590080      activation_72[0][0]              
__________________________________________________________________________________________________
bn4a_branch2b (BatchNormalizati (None, 14, 14, 256)  1024        res4a_branch2b[0][0]             
__________________________________________________________________________________________________
activation_73 (Activation)      (None, 14, 14, 256)  0           bn4a_branch2b[0][0]              
__________________________________________________________________________________________________
res4a_branch2c (Conv2D)         (None, 14, 14, 1024) 263168      activation_73[0][0]              
__________________________________________________________________________________________________
res4a_branch1 (Conv2D)          (None, 14, 14, 1024) 525312      activation_71[0][0]              
__________________________________________________________________________________________________
bn4a_branch2c (BatchNormalizati (None, 14, 14, 1024) 4096        res4a_branch2c[0][0]             
__________________________________________________________________________________________________
bn4a_branch1 (BatchNormalizatio (None, 14, 14, 1024) 4096        res4a_branch1[0][0]              
__________________________________________________________________________________________________
add_24 (Add)                    (None, 14, 14, 1024) 0           bn4a_branch2c[0][0]              
                                                                 bn4a_branch1[0][0]               
__________________________________________________________________________________________________
activation_74 (Activation)      (None, 14, 14, 1024) 0           add_24[0][0]                     
__________________________________________________________________________________________________
res4b_branch2a (Conv2D)         (None, 14, 14, 256)  262400      activation_74[0][0]              
__________________________________________________________________________________________________
bn4b_branch2a (BatchNormalizati (None, 14, 14, 256)  1024        res4b_branch2a[0][0]             
__________________________________________________________________________________________________
activation_75 (Activation)      (None, 14, 14, 256)  0           bn4b_branch2a[0][0]              
__________________________________________________________________________________________________
res4b_branch2b (Conv2D)         (None, 14, 14, 256)  590080      activation_75[0][0]              
__________________________________________________________________________________________________
bn4b_branch2b (BatchNormalizati (None, 14, 14, 256)  1024        res4b_branch2b[0][0]             
__________________________________________________________________________________________________
activation_76 (Activation)      (None, 14, 14, 256)  0           bn4b_branch2b[0][0]              
__________________________________________________________________________________________________
res4b_branch2c (Conv2D)         (None, 14, 14, 1024) 263168      activation_76[0][0]              
__________________________________________________________________________________________________
bn4b_branch2c (BatchNormalizati (None, 14, 14, 1024) 4096        res4b_branch2c[0][0]             
__________________________________________________________________________________________________
add_25 (Add)                    (None, 14, 14, 1024) 0           bn4b_branch2c[0][0]              
                                                                 activation_74[0][0]              
__________________________________________________________________________________________________
activation_77 (Activation)      (None, 14, 14, 1024) 0           add_25[0][0]                     
__________________________________________________________________________________________________
res4c_branch2a (Conv2D)         (None, 14, 14, 256)  262400      activation_77[0][0]              
__________________________________________________________________________________________________
bn4c_branch2a (BatchNormalizati (None, 14, 14, 256)  1024        res4c_branch2a[0][0]             
__________________________________________________________________________________________________
activation_78 (Activation)      (None, 14, 14, 256)  0           bn4c_branch2a[0][0]              
__________________________________________________________________________________________________
res4c_branch2b (Conv2D)         (None, 14, 14, 256)  590080      activation_78[0][0]              
__________________________________________________________________________________________________
bn4c_branch2b (BatchNormalizati (None, 14, 14, 256)  1024        res4c_branch2b[0][0]             
__________________________________________________________________________________________________
activation_79 (Activation)      (None, 14, 14, 256)  0           bn4c_branch2b[0][0]              
__________________________________________________________________________________________________
res4c_branch2c (Conv2D)         (None, 14, 14, 1024) 263168      activation_79[0][0]              
__________________________________________________________________________________________________
bn4c_branch2c (BatchNormalizati (None, 14, 14, 1024) 4096        res4c_branch2c[0][0]             
__________________________________________________________________________________________________
add_26 (Add)                    (None, 14, 14, 1024) 0           bn4c_branch2c[0][0]              
                                                                 activation_77[0][0]              
__________________________________________________________________________________________________
activation_80 (Activation)      (None, 14, 14, 1024) 0           add_26[0][0]                     
__________________________________________________________________________________________________
res4d_branch2a (Conv2D)         (None, 14, 14, 256)  262400      activation_80[0][0]              
__________________________________________________________________________________________________
bn4d_branch2a (BatchNormalizati (None, 14, 14, 256)  1024        res4d_branch2a[0][0]             
__________________________________________________________________________________________________
activation_81 (Activation)      (None, 14, 14, 256)  0           bn4d_branch2a[0][0]              
__________________________________________________________________________________________________
res4d_branch2b (Conv2D)         (None, 14, 14, 256)  590080      activation_81[0][0]              
__________________________________________________________________________________________________
bn4d_branch2b (BatchNormalizati (None, 14, 14, 256)  1024        res4d_branch2b[0][0]             
__________________________________________________________________________________________________
activation_82 (Activation)      (None, 14, 14, 256)  0           bn4d_branch2b[0][0]              
__________________________________________________________________________________________________
res4d_branch2c (Conv2D)         (None, 14, 14, 1024) 263168      activation_82[0][0]              
__________________________________________________________________________________________________
bn4d_branch2c (BatchNormalizati (None, 14, 14, 1024) 4096        res4d_branch2c[0][0]             
__________________________________________________________________________________________________
add_27 (Add)                    (None, 14, 14, 1024) 0           bn4d_branch2c[0][0]              
                                                                 activation_80[0][0]              
__________________________________________________________________________________________________
activation_83 (Activation)      (None, 14, 14, 1024) 0           add_27[0][0]                     
__________________________________________________________________________________________________
res4e_branch2a (Conv2D)         (None, 14, 14, 256)  262400      activation_83[0][0]              
__________________________________________________________________________________________________
bn4e_branch2a (BatchNormalizati (None, 14, 14, 256)  1024        res4e_branch2a[0][0]             
__________________________________________________________________________________________________
activation_84 (Activation)      (None, 14, 14, 256)  0           bn4e_branch2a[0][0]              
__________________________________________________________________________________________________
res4e_branch2b (Conv2D)         (None, 14, 14, 256)  590080      activation_84[0][0]              
__________________________________________________________________________________________________
bn4e_branch2b (BatchNormalizati (None, 14, 14, 256)  1024        res4e_branch2b[0][0]             
__________________________________________________________________________________________________
activation_85 (Activation)      (None, 14, 14, 256)  0           bn4e_branch2b[0][0]              
__________________________________________________________________________________________________
res4e_branch2c (Conv2D)         (None, 14, 14, 1024) 263168      activation_85[0][0]              
__________________________________________________________________________________________________
bn4e_branch2c (BatchNormalizati (None, 14, 14, 1024) 4096        res4e_branch2c[0][0]             
__________________________________________________________________________________________________
add_28 (Add)                    (None, 14, 14, 1024) 0           bn4e_branch2c[0][0]              
                                                                 activation_83[0][0]              
__________________________________________________________________________________________________
activation_86 (Activation)      (None, 14, 14, 1024) 0           add_28[0][0]                     
__________________________________________________________________________________________________
res4f_branch2a (Conv2D)         (None, 14, 14, 256)  262400      activation_86[0][0]              
__________________________________________________________________________________________________
bn4f_branch2a (BatchNormalizati (None, 14, 14, 256)  1024        res4f_branch2a[0][0]             
__________________________________________________________________________________________________
activation_87 (Activation)      (None, 14, 14, 256)  0           bn4f_branch2a[0][0]              
__________________________________________________________________________________________________
res4f_branch2b (Conv2D)         (None, 14, 14, 256)  590080      activation_87[0][0]              
__________________________________________________________________________________________________
bn4f_branch2b (BatchNormalizati (None, 14, 14, 256)  1024        res4f_branch2b[0][0]             
__________________________________________________________________________________________________
activation_88 (Activation)      (None, 14, 14, 256)  0           bn4f_branch2b[0][0]              
__________________________________________________________________________________________________
res4f_branch2c (Conv2D)         (None, 14, 14, 1024) 263168      activation_88[0][0]              
__________________________________________________________________________________________________
bn4f_branch2c (BatchNormalizati (None, 14, 14, 1024) 4096        res4f_branch2c[0][0]             
__________________________________________________________________________________________________
add_29 (Add)                    (None, 14, 14, 1024) 0           bn4f_branch2c[0][0]              
                                                                 activation_86[0][0]              
__________________________________________________________________________________________________
activation_89 (Activation)      (None, 14, 14, 1024) 0           add_29[0][0]                     
__________________________________________________________________________________________________
res5a_branch2a (Conv2D)         (None, 7, 7, 512)    524800      activation_89[0][0]              
__________________________________________________________________________________________________
bn5a_branch2a (BatchNormalizati (None, 7, 7, 512)    2048        res5a_branch2a[0][0]             
__________________________________________________________________________________________________
activation_90 (Activation)      (None, 7, 7, 512)    0           bn5a_branch2a[0][0]              
__________________________________________________________________________________________________
res5a_branch2b (Conv2D)         (None, 7, 7, 512)    2359808     activation_90[0][0]              
__________________________________________________________________________________________________
bn5a_branch2b (BatchNormalizati (None, 7, 7, 512)    2048        res5a_branch2b[0][0]             
__________________________________________________________________________________________________
activation_91 (Activation)      (None, 7, 7, 512)    0           bn5a_branch2b[0][0]              
__________________________________________________________________________________________________
res5a_branch2c (Conv2D)         (None, 7, 7, 2048)   1050624     activation_91[0][0]              
__________________________________________________________________________________________________
res5a_branch1 (Conv2D)          (None, 7, 7, 2048)   2099200     activation_89[0][0]              
__________________________________________________________________________________________________
bn5a_branch2c (BatchNormalizati (None, 7, 7, 2048)   8192        res5a_branch2c[0][0]             
__________________________________________________________________________________________________
bn5a_branch1 (BatchNormalizatio (None, 7, 7, 2048)   8192        res5a_branch1[0][0]              
__________________________________________________________________________________________________
add_30 (Add)                    (None, 7, 7, 2048)   0           bn5a_branch2c[0][0]              
                                                                 bn5a_branch1[0][0]               
__________________________________________________________________________________________________
activation_92 (Activation)      (None, 7, 7, 2048)   0           add_30[0][0]                     
__________________________________________________________________________________________________
res5b_branch2a (Conv2D)         (None, 7, 7, 512)    1049088     activation_92[0][0]              
__________________________________________________________________________________________________
bn5b_branch2a (BatchNormalizati (None, 7, 7, 512)    2048        res5b_branch2a[0][0]             
__________________________________________________________________________________________________
activation_93 (Activation)      (None, 7, 7, 512)    0           bn5b_branch2a[0][0]              
__________________________________________________________________________________________________
res5b_branch2b (Conv2D)         (None, 7, 7, 512)    2359808     activation_93[0][0]              
__________________________________________________________________________________________________
bn5b_branch2b (BatchNormalizati (None, 7, 7, 512)    2048        res5b_branch2b[0][0]             
__________________________________________________________________________________________________
activation_94 (Activation)      (None, 7, 7, 512)    0           bn5b_branch2b[0][0]              
__________________________________________________________________________________________________
res5b_branch2c (Conv2D)         (None, 7, 7, 2048)   1050624     activation_94[0][0]              
__________________________________________________________________________________________________
bn5b_branch2c (BatchNormalizati (None, 7, 7, 2048)   8192        res5b_branch2c[0][0]             
__________________________________________________________________________________________________
add_31 (Add)                    (None, 7, 7, 2048)   0           bn5b_branch2c[0][0]              
                                                                 activation_92[0][0]              
__________________________________________________________________________________________________
activation_95 (Activation)      (None, 7, 7, 2048)   0           add_31[0][0]                     
__________________________________________________________________________________________________
res5c_branch2a (Conv2D)         (None, 7, 7, 512)    1049088     activation_95[0][0]              
__________________________________________________________________________________________________
bn5c_branch2a (BatchNormalizati (None, 7, 7, 512)    2048        res5c_branch2a[0][0]             
__________________________________________________________________________________________________
activation_96 (Activation)      (None, 7, 7, 512)    0           bn5c_branch2a[0][0]              
__________________________________________________________________________________________________
res5c_branch2b (Conv2D)         (None, 7, 7, 512)    2359808     activation_96[0][0]              
__________________________________________________________________________________________________
bn5c_branch2b (BatchNormalizati (None, 7, 7, 512)    2048        res5c_branch2b[0][0]             
__________________________________________________________________________________________________
activation_97 (Activation)      (None, 7, 7, 512)    0           bn5c_branch2b[0][0]              
__________________________________________________________________________________________________
res5c_branch2c (Conv2D)         (None, 7, 7, 2048)   1050624     activation_97[0][0]              
__________________________________________________________________________________________________
bn5c_branch2c (BatchNormalizati (None, 7, 7, 2048)   8192        res5c_branch2c[0][0]             
__________________________________________________________________________________________________
add_32 (Add)                    (None, 7, 7, 2048)   0           bn5c_branch2c[0][0]              
                                                                 activation_95[0][0]              
__________________________________________________________________________________________________
activation_98 (Activation)      (None, 7, 7, 2048)   0           add_32[0][0]                     
__________________________________________________________________________________________________
global_average_pooling2d_2 (Glo (None, 2048)         0           activation_98[0][0]              
__________________________________________________________________________________________________
dropout_3 (Dropout)             (None, 2048)         0           global_average_pooling2d_2[0][0] 
__________________________________________________________________________________________________
dense_3 (Dense)                 (None, 4096)         8392704     dropout_3[0][0]                  
__________________________________________________________________________________________________
batch_normalization_2 (BatchNor (None, 4096)         16384       dense_3[0][0]                    
__________________________________________________________________________________________________
dropout_4 (Dropout)             (None, 4096)         0           batch_normalization_2[0][0]      
__________________________________________________________________________________________________
dense_4 (Dense)                 (None, 5004)         20501388    dropout_4[0][0]                  
==================================================================================================
Total params: 52,498,188
Trainable params: 28,955,404
Non-trainable params: 23,542,784
__________________________________________________________________________________________________
WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_ops.py:3066: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.
Epoch 1/50
474/474 [==============================] - 519s 1s/step - loss: 8.4220 - acc: 0.0276 - val_loss: 5.8134 - val_acc: 0.1404

Epoch 00001: val_acc improved from -inf to 0.14042, saving model to ./drive/My Drive/models/without_whale/without_whalev5/01-8.42-5.81.hdf5
Epoch 2/50
474/474 [==============================] - 502s 1s/step - loss: 7.2171 - acc: 0.0809 - val_loss: 4.3133 - val_acc: 0.2817

Epoch 00002: val_acc improved from 0.14042 to 0.28169, saving model to ./drive/My Drive/models/without_whale/without_whalev5/02-7.22-4.31.hdf5
Epoch 3/50
474/474 [==============================] - 505s 1s/step - loss: 5.8098 - acc: 0.1457 - val_loss: 3.6735 - val_acc: 0.3521

Epoch 00003: val_acc improved from 0.28169 to 0.35211, saving model to ./drive/My Drive/models/without_whale/without_whalev5/03-5.81-3.67.hdf5
Epoch 4/50
474/474 [==============================] - 505s 1s/step - loss: 4.6063 - acc: 0.2225 - val_loss: 3.0987 - val_acc: 0.4256

Epoch 00004: val_acc improved from 0.35211 to 0.42555, saving model to ./drive/My Drive/models/without_whale/without_whalev5/04-4.61-3.10.hdf5
Epoch 5/50
474/474 [==============================] - 508s 1s/step - loss: 3.5719 - acc: 0.3203 - val_loss: 3.0616 - val_acc: 0.4467

Epoch 00005: val_acc improved from 0.42555 to 0.44668, saving model to ./drive/My Drive/models/without_whale/without_whalev5/05-3.57-3.06.hdf5
Epoch 6/50
474/474 [==============================] - 509s 1s/step - loss: 2.8159 - acc: 0.4101 - val_loss: 2.9259 - val_acc: 0.4678

Epoch 00006: val_acc improved from 0.44668 to 0.46781, saving model to ./drive/My Drive/models/without_whale/without_whalev5/06-2.82-2.93.hdf5
Epoch 7/50
474/474 [==============================] - 509s 1s/step - loss: 2.2522 - acc: 0.4959 - val_loss: 3.1489 - val_acc: 0.4624

Epoch 00007: val_acc did not improve from 0.46781
Epoch 8/50
474/474 [==============================] - 505s 1s/step - loss: 1.8516 - acc: 0.5733 - val_loss: 2.8370 - val_acc: 0.4899

Epoch 00008: val_acc improved from 0.46781 to 0.48994, saving model to ./drive/My Drive/models/without_whale/without_whalev5/08-1.85-2.84.hdf5
Epoch 9/50
474/474 [==============================] - 506s 1s/step - loss: 1.6278 - acc: 0.6124 - val_loss: 2.9060 - val_acc: 0.5161

Epoch 00009: val_acc improved from 0.48994 to 0.51610, saving model to ./drive/My Drive/models/without_whale/without_whalev5/09-1.63-2.91.hdf5
Epoch 10/50
474/474 [==============================] - 506s 1s/step - loss: 1.4497 - acc: 0.6487 - val_loss: 2.6826 - val_acc: 0.5382

Epoch 00010: val_acc improved from 0.51610 to 0.53823, saving model to ./drive/My Drive/models/without_whale/without_whalev5/10-1.45-2.68.hdf5
Epoch 11/50
474/474 [==============================] - 505s 1s/step - loss: 1.3285 - acc: 0.6798 - val_loss: 2.7215 - val_acc: 0.5553

Epoch 00011: val_acc improved from 0.53823 to 0.55533, saving model to ./drive/My Drive/models/without_whale/without_whalev5/11-1.33-2.72.hdf5
Epoch 12/50
474/474 [==============================] - 505s 1s/step - loss: 1.1996 - acc: 0.7073 - val_loss: 2.8060 - val_acc: 0.5221

Epoch 00012: val_acc did not improve from 0.55533
Epoch 13/50
474/474 [==============================] - 505s 1s/step - loss: 1.1268 - acc: 0.7178 - val_loss: 2.7300 - val_acc: 0.5744

Epoch 00013: val_acc improved from 0.55533 to 0.57445, saving model to ./drive/My Drive/models/without_whale/without_whalev5/13-1.13-2.73.hdf5
Epoch 14/50
474/474 [==============================] - 505s 1s/step - loss: 1.0509 - acc: 0.7389 - val_loss: 2.6993 - val_acc: 0.5657

Epoch 00014: val_acc did not improve from 0.57445
Epoch 15/50
474/474 [==============================] - 504s 1s/step - loss: 0.9908 - acc: 0.7507 - val_loss: 2.8211 - val_acc: 0.5614

Epoch 00015: val_acc did not improve from 0.57445
Epoch 16/50
474/474 [==============================] - 505s 1s/step - loss: 0.9159 - acc: 0.7688 - val_loss: 2.8153 - val_acc: 0.5714

Epoch 00016: val_acc did not improve from 0.57445
Epoch 17/50
474/474 [==============================] - 506s 1s/step - loss: 0.8659 - acc: 0.7771 - val_loss: 2.8439 - val_acc: 0.5946

Epoch 00017: val_acc improved from 0.57445 to 0.59457, saving model to ./drive/My Drive/models/without_whale/without_whalev5/17-0.87-2.84.hdf5
Epoch 18/50
474/474 [==============================] - 505s 1s/step - loss: 0.8075 - acc: 0.7923 - val_loss: 2.6584 - val_acc: 0.5724

Epoch 00018: val_acc did not improve from 0.59457
Epoch 19/50
474/474 [==============================] - 505s 1s/step - loss: 0.8037 - acc: 0.7930 - val_loss: 2.6727 - val_acc: 0.5744

Epoch 00019: val_acc did not improve from 0.59457
Epoch 20/50
474/474 [==============================] - 506s 1s/step - loss: 0.7386 - acc: 0.8080 - val_loss: 2.6608 - val_acc: 0.6006

Epoch 00020: val_acc improved from 0.59457 to 0.60060, saving model to ./drive/My Drive/models/without_whale/without_whalev5/20-0.74-2.66.hdf5
Epoch 21/50
474/474 [==============================] - 506s 1s/step - loss: 0.7272 - acc: 0.8117 - val_loss: 2.6613 - val_acc: 0.6118

Epoch 00021: val_acc improved from 0.60060 to 0.61184, saving model to ./drive/My Drive/models/without_whale/without_whalev5/21-0.73-2.66.hdf5
Epoch 22/50
 70/474 [===>..........................] - ETA: 6:23 - loss: 0.5654 - acc: 0.8438```



In [0]:
from keras import regularizers
class ModelV7():
    def __init__(self):
        self.name = 'v7'
    def get_model(self):
        conv_base = resnet50.ResNet50(weights='imagenet',
                                      include_top=False,
                                      input_shape=(input_size, input_size, 3))
        for layer in conv_base.layers[:]:
            if 'BatchNormalization' in str(layer):
                layer.trainable = True
            else:
                layer.trainable = False
        main_input = conv_base.input
        embedding = conv_base.output
        x = layers.GlobalMaxPooling2D()(embedding)
        x = layers.Dropout(0.5)(x)
        x = layers.Dense(4096, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.5)(x)
        x = layers.Dense(5004, activation='softmax')(x)
        model = models.Model(inputs=[main_input], outputs=[x])
        model.compile(
            loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
        return model


In [0]:
from keras import regularizers
class ModelV7():
    def __init__(self):
        self.name = 'v7'
    def get_model(self):
        conv_base = resnet50.ResNet50(weights='imagenet',
                                      include_top=False,
                                      input_shape=(input_size, input_size, 3))
        for layer in conv_base.layers[:]:
            if 'BatchNormalization' in str(layer):
                layer.trainable = True
            else:
                layer.trainable = False
        main_input = conv_base.input
        embedding = conv_base.output
        x = layers.GlobalMaxPooling2D()(embedding)
        x = layers.Dropout(0.5)(x)
        x = layers.Dense(4096, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.5)(x)
        x = layers.Dense(5004, activation='softmax')(x)
        model = models.Model(inputs=[main_input], outputs=[x])
        model.compile(
            loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
        return model
     

In [0]:
model = models.load_model('drive/My Drive/models/without_whale/without_whalev7/40-5.02-5.15.hdf5')


In [0]:
INITIAL_EPOCH = 90
epochs = 100
train_generator, val_generator = load_data()
model_wrapper = ModelV7()
# model = model_wrapper.get_model()
model.summary()

model_dir = os.path.join(model_base_dir, 'without_whale' + model_wrapper.name)
model_dir = os.path.join('./drive/My Drive', model_dir)

os.makedirs(model_dir, exist_ok=True)

model_checkpoint_path = os.path.join(model_dir, '{epoch:02d}-{loss:.2f}-{val_loss:.2f}.hdf5')
model_checkpoint = ModelCheckpoint(model_checkpoint_path, monitor='val_acc', verbose=1, save_best_only=True)
tensor_board = TensorBoard(log_dir=os.path.join(model_dir))
history = model.fit_generator(train_generator,
                    steps_per_epoch=len(train_generator),
                    epochs=epochs,
                    validation_data=val_generator,
                    validation_steps=50,
                    callbacks=[model_checkpoint, tensor_board],
                    initial_epoch=INITIAL_EPOCH         
                    )

Found 15160 images belonging to 5004 classes.
Found 537 images belonging to 5004 classes.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
___________________

追加学習 20 epoch with unfreeze top layers

In [0]:
for layer in model.layers[-17:]:
  layer.trainable=True

In [0]:
model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(lr=0.0001), metrics=['acc'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [0]:
epochs = 120
INITIAL_EPOCH = 100
history = model.fit_generator(train_generator,
                    steps_per_epoch=len(train_generator),
                    epochs=epochs,
                    validation_data=val_generator,
                    validation_steps=50,
                    callbacks=[model_checkpoint, tensor_board],
                    initial_epoch=INITIAL_EPOCH         
                    )

Epoch 101/120
474/474 [==============================] - 515s 1s/step - loss: 3.2689 - acc: 0.7732 - val_loss: 3.2029 - val_acc: 0.7938

Epoch 00101: val_acc improved from 0.76660 to 0.79376, saving model to ./drive/My Drive/models/without_whale/without_whalev7/101-3.27-3.20.hdf5
Epoch 102/120
474/474 [==============================] - 497s 1s/step - loss: 2.6575 - acc: 0.8201 - val_loss: 2.9514 - val_acc: 0.7907

Epoch 00102: val_acc did not improve from 0.79376
Epoch 103/120
474/474 [==============================] - 490s 1s/step - loss: 2.2567 - acc: 0.8461 - val_loss: 2.6104 - val_acc: 0.8149

Epoch 00103: val_acc improved from 0.79376 to 0.81489, saving model to ./drive/My Drive/models/without_whale/without_whalev7/103-2.26-2.61.hdf5
Epoch 104/120
474/474 [==============================] - 493s 1s/step - loss: 1.9269 - acc: 0.8681 - val_loss: 2.3060 - val_acc: 0.8099

Epoch 00104: val_acc did not improve from 0.81489
Epoch 105/120
474/474 [==============================] - 486s 1s

V8


In [0]:
from keras import regularizers
class ModelV8():
    def __init__(self):
        self.name = 'v8'
    def init_model(self):
        conv_base = resnet50.ResNet50(weights='imagenet',
                                      include_top=False,
                                      input_shape=(input_size, input_size, 3))
        for layer in conv_base.layers[:]:
            if 'BatchNormalization' in str(layer):
                layer.trainable = True
            else:
                layer.trainable = False
        main_input = conv_base.input
        embedding = conv_base.output
        x = layers.GlobalMaxPooling2D()(embedding)
        x = layers.Dropout(0.5)(x)
        x = layers.Dense(4096, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.5)(x)
        x = layers.Dense(5004, activation='softmax')(x)
        model = models.Model(inputs=[main_input], outputs=[x])
        model.compile(
            loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
        self.model = model
        return model
    def train(self, train_generator, val_generator, callbacks, epochs=100, initial_epoch=None):
        epochs1 = 10
        if initial_epoch is None:
            initial_epochs = 0
        if initial_epoch < epochs1:
            inicial_epoch2 = epochs1
        else:
            initial_epoch2 = initial_epoch
        self.model.fit_generator(
            train_generator,
            steps_per_epoch=len(train_generator),
            epochs=epochs1,
            validation_data=val_generator,
            validation_steps=50,
            callbacks=callbacks,
            initial_epoch=initial_epoch
        )
        for layer in self.model.layers[-17:]:
            layer.trainable=True
        self.model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(lr=0.001), metrics=['acc'])
        self.model.fit_generator(
            train_generator,
            steps_per_epoch=len(train_generator),
            epochs=epochs,
            validation_data=val_generator,
            validation_steps=50,
            callbacks=callbacks,
            initial_epoch=initial_epoch,
        )



In [0]:
train_generator, val_generator = load_data()
model_wrapper = ModelV8()
model_wrapper.init_model()
model_wrapper.model.summary()

model_dir = os.path.join(model_base_dir, 'without_whale' + model_wrapper.name)
model_dir = os.path.join('./drive/My Drive', model_dir)

os.makedirs(model_dir, exist_ok=True)

model_checkpoint_path = os.path.join(model_dir, 'model.hdf5')
model_checkpoint = ModelCheckpoint(model_checkpoint_path, monitor='val_acc', verbose=1, save_best_only=True)
tensor_board = TensorBoard(log_dir=os.path.join(model_dir))
model_wrapper.train(train_generator, val_generator, [model_checkpoint, tensor_board])


Found 15160 images belonging to 5004 classes.
Found 537 images belonging to 5004 classes.


/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [8]:
train_generator, val_generator = load_data()

model_wrapper = ModelV8()
model_dir = os.path.join(model_base_dir, 'without_whale' + model_wrapper.name)
model_dir = os.path.join('./drive/My Drive', model_dir)
model = models.load_model(os.path.join(model_dir, '98epochs_3.2204.hdf5'))
model_wrapper.model = model

model_checkpoint_path = os.path.join(model_dir, 'model.hdf5')
model_checkpoint = ModelCheckpoint(model_checkpoint_path, monitor='val_acc', verbose=1, save_best_only=True)
tensor_board = TensorBoard(log_dir=os.path.join(model_dir))

model_wrapper.train(train_generator, val_generator, [model_checkpoint, tensor_board], epochs= 150, initial_epoch=99)
model.save(os.path.join(model_dir, str(150) + 'model.hdf5'))


Found 15160 images belonging to 5004 classes.
Found 537 images belonging to 5004 classes.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 100/150
474/474 [==============================] - 530s 1s/step - loss: 3.3748 - acc: 0.8145 - val_loss: 4.6262 - val_acc: 0.7282

Epoch 00100: val_acc improved from -inf to 0.72818, saving model to ./drive/My Drive/models/without_whale/without_whalev8/model.hdf5
Epoch 101/150
474/474 [==============================] - 487s 1s/step - loss: 3.4094 - acc: 0.8210 - val_loss: 4.2187 - val_acc: 0.7545

Epoch 00101: val_acc improved from 0.72818 to 0.75453, saving model to ./drive/My Drive/models/without_whale/without_whalev8/model.hdf5
Epoch 102/150
474/474 [==============================]

KeyboardInterrupt: ignored

# 新しいセクション